In [35]:


##### import pandas as pd
import seaborn as sns
import numpy as np

import pandas as pd
from fuzzywuzzy import process


#upload raw data from inline questionnaire
raw_onlinefb=pd.read_excel("online_douman_raw.xlsx")
#drop the first unwanted row
raw_onlinefb=raw_onlinefb.drop([0], axis=0)

#drop unwanted columns
raw_onlinefb=raw_onlinefb.drop([ 'EndDate', 'Status', 'IPAddress','Finished', 
       'RecipientLastName', 'RecordedDate', 'ResponseId', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 
                                'LocationLatitude',
                                'LocationLongitude', 'UserLanguage',], axis=1)

#drop the participants did not complete their survey

raw_onlinefb=raw_onlinefb[raw_onlinefb['Progress']==100] 
distribution=['anonymous','qr']
raw_onlinefb=raw_onlinefb.loc[raw_onlinefb['DistributionChannel'].isin(distribution)]

#drop the cols of VP codes if they are empty
raw_onlinefb.dropna(subset=['Q132_1'], inplace=True)
#rename the code in Q132-1 which is for Gato14KO which has been wrongly filled as ANPE07B

#columns that cover TICS QS
df_tics=['Q2_1', 'Q2_2', 'Q2_3', 'Q2_4', 'Q2_5', 'Q111_1', 'Q111_2', 'Q111_3', 'Q111_4', 'Q111_5',
         'Q76_1', 'Q76_2','Q76_3', 'Q76_5', 'Q110_1', 'Q110_2', 'Q110_3', 'Q110_4', 
         'Q110_5', 'Q77_1', 'Q77_2', 'Q77_3', 'Q77_4', 'Q77_5', 'Q112_1',  
         'Q112_2', 'Q112_3', 'Q112_4', 'Q112_5', 'Q78_1', 'Q78_2','Q78_3', 
         'Q78_4', 'Q78_5', 'Q113_1', 'Q113_2', 'Q113_3', 'Q113_4',
         'Q113_5', 'Q79_1', 'Q79_2', 'Q79_3', 'Q79_4', 'Q114_1','Q114_2', 
         'Q114_3', 'Q114_4', 'Q114_5', 'Q80_1',  'Q80_3',
         'Q80_4', 'Q80_5', 'Q115_1', 'Q115_2', 'Q115_3', 'Q115_4',
         'Q115_5']

#There are three control question that 
#every VP need to have three of that in his/her DF
control_qs=['Q76_4', 'Q79_5','Q80_2',]
Control_1=[]
Control_2=[]
Control_3=[]
for i in raw_onlinefb[control_qs[0]]:
    if i==4:
        a='True'
        Control_1.append(a)
    else:
        
        b='false'
        Control_1.append(b)
    
for i in raw_onlinefb[control_qs[1]]:
    if i==1:
        a='True'
        Control_2.append(a)
    else:
        
        b='False'
        Control_2.append(b)   
for i in raw_onlinefb[control_qs[2]]:
    if i==2:
        a='True'
        Control_3.append(a)
    else:
        
        b='False'
        Control_3.append(b)    

#identifying the 9-subgroups of TICS
#category and its questions
#items covering work overload
WOL=[1,4,44,54,17,27,38,50]
#items covering social overload
SO=[49,57,7,19,28,39]
#items covering Pressure to Perform
PP=[8,12,14,22,30,23,32,40,43]
#items covering work discontent
WD=[5,10,13,41,21,37,48,53]
#items covering Excessive Demands at Work
EDW=[3,47,20,24,35,55]
#items covering Lack of Social Recognition
LSR=[2,18,31,46]
#items covering Social Tensions
ST=[6,33,15,26,45,52]
#items covering social isolation
SI=[29,34,11,42,51,56]
#chronic worring
CW=[9,16,25,36]
tics_num=[]
for i in range(1,61):
    n=i
    if n in LSR:
        n=str(n)+"_"+'LSR'
        tics_num.append(n)
    elif n in SO:
        n=str(n)+"_"+'SO'
        tics_num.append(n)
    elif n in PP:
        n=str(n)+"_"+'PP'
        tics_num.append(n)
    elif n in WD:
        n=str(n)+"_"+'WD'
        tics_num.append(n)
    elif n in EDW:
        n=str(n)+"_"+'EDW'
        tics_num.append(n)
    elif n in WOL:
        n=str(n)+"_"+'WOL'
        tics_num.append(n)
    elif n in ST:
        n=str(n)+"_"+'ST'
        tics_num.append(n)
    elif n in SI:
        n=str(n)+"_"+'SI'
        tics_num.append(n)
    elif n in CW:
        n=str(n)+"_"+'CW'
        tics_num.append(n)
        
#Convert label of tic cols into numbers
#with shoeing they belong to which subcategory of tics
tics_dic=dict(zip(df_tics,tics_num))   
raw_onlinefb=raw_onlinefb.rename(columns={'Q2_1': '1_WOL', 'Q2_2': '2_LSR' ,'Q2_3': '3_EDW', 'Q2_4': '4_WOL',
                                          'Q2_5': '5_WD',
 'Q111_1': '6_ST', 'Q111_2': '7_SO', 'Q111_3': '8_PP', 'Q111_4': '9_CW', 'Q111_5': '10_WD', 'Q76_1': '11_SI', 'Q76_2': '12_PP', 'Q76_3': '13_WD',
 'Q76_5': '14_PP',
 'Q110_1': '15_ST', 'Q110_2': '16_CW', 'Q110_3': '17_WOL', 'Q110_4': '18_LSR', 'Q110_5': '19_SO', 'Q77_1': '20_EDW', 'Q77_2': '21_WD', 'Q77_3': '22_PP', 'Q77_4': '23_PP', 'Q77_5': '24_EDW', 'Q112_1': '25_CW', 'Q112_2': '26_ST', 'Q112_3': '27_WOL', 'Q112_4': '28_SO', 'Q112_5': '29_SI', 'Q78_1': '30_PP',
 'Q78_2': '31_LSR',
 'Q78_3': '32_PP',
 'Q78_4': '33_ST', 'Q78_5': '34_SI', 'Q113_1': '35_EDW', 'Q113_2': '36_CW', 'Q113_3': '37_WD', 'Q113_4': '38_WOL', 'Q113_5': '39_SO', 'Q79_1': '40_PP', 'Q79_2': '41_WD',
 'Q79_3': '42_SI', 'Q79_4': '43_PP', 'Q114_1': '44_WOL', 'Q114_2': '45_ST', 'Q114_3': '46_LSR', 'Q114_4': '47_EDW', 'Q114_5': '48_WD',
 'Q80_1': '49_SO', 'Q80_3': '50_WOL', 'Q80_4': '51_SI', 'Q80_5': '52_ST',
 'Q115_1': '53_WD', 'Q115_2': '54_WOL', 'Q115_3': '55_EDW', 'Q115_4': '56_SI', 'Q115_5': '57_SO'})
raw_onlinefb['TICS']=raw_onlinefb[tics_num].sum(axis=1)
raw_onlinefb['TICS_control1']=Control_1
raw_onlinefb['TICS_control2']=Control_2
raw_onlinefb['TICS_control3']=Control_3


# In[26]:



#SVO data preparation
#to calculate the social value orientation
svo=['Q92_1','Q103_1','Q102_1','Q101_1','Q100_1','Q99_1','Q98_1','Q97_1','Q96_1',]
svo_num=['svo_1','svo_2','svo_3','svo_4','svo_5','svo_6','svo_7','svo_8','svo_9']
svo_dict=dict(zip(svo,svo_num))
raw_onlinefb=raw_onlinefb.rename(columns={'Q92_1': 'svo_1',
 'Q103_1': 'svo_2',
 'Q102_1': 'svo_3',
 'Q101_1': 'svo_4',
 'Q100_1': 'svo_5',
 'Q99_1': 'svo_6',
 'Q98_1': 'svo_7',
 'Q97_1': 'svo_8',
 'Q96_1': 'svo_9'})
#individualistic is H
#Competitive Com
# properative pro
#the order of orientations in online qs of SVO
#I-Com-pro	Com-pro-I	pro-I-Com	I-Com-pro	Com-pro-I
#pro-I-Com	pro-Com-I	Com-I-pro	I-pro-Com
#0 means pro/1 means Com/ 2 means I
#I-Com-pro
#the empty rows are called NONE
svo_1=[]
for i in raw_onlinefb['svo_1']:
    if i==1:
        svo_q1='ind'
        svo_1.append(svo_q1)
    elif i==2:
        
        svo_q1='Com'
        svo_1.append(svo_q1)
    elif i==3:
        
        svo_q1='pro'
        svo_1.append(svo_q1)  
    else:
        svo_q1='None'
        svo_1.append(svo_q1)
        
#add the svo_1 col
raw_onlinefb['svo_1']=svo_1
#Com-pro-I       
#svo_q2
svo_2=[]
for i in raw_onlinefb['svo_2']:
    if i==1:
        svo_q2='Com'
        svo_2.append(svo_q2)
    elif i==2:
        
        svo_q2='pro'
        svo_2.append(svo_q2)
    elif i==3:
        
        svo_q2='ind'
        svo_2.append(svo_q2) 
    else:
        svo_q2='None'
        svo_2.append(svo_q2)
        
    
        
#add the svo_2 col
raw_onlinefb['svo_2']=svo_2
#pro-I-Com
#svo_q3
svo_3=[]
for i in raw_onlinefb['svo_3']:
    if i==1:
        svo_q3='pro'
        svo_3.append(svo_q3)
    elif i==2:
        
        svo_q3='ind'
        svo_3.append(svo_q3)
    elif i==3:
        
        svo_q3='Com'
        svo_3.append(svo_q3)
    else:
        svo_q3='None'
        svo_3.append(svo_q3)
        
#add the svo_3 col
raw_onlinefb['svo_3']=svo_3

#I-Com-pro
svo_4=[]
for i in raw_onlinefb['svo_4']:
    if i==1:
        svo_q4='ind'
        svo_4.append(svo_q4)
    elif i==2:
        
        svo_q4='Com'
        svo_4.append(svo_q4)
    
    elif i==3:
        
        svo_q4='pro'
        svo_4.append(svo_q4) 
    else:
        svo_q4='None'
        svo_4.append(svo_q4)
        
#add the svo_4 col
raw_onlinefb['svo_4']=svo_4  


#Com-pro-I       
#svo_q5
svo_5=[]
for i in raw_onlinefb['svo_5']:
    if i==1:
        svo_q5='Com'
        svo_5.append(svo_q5)
    elif i==2:
        
        svo_q5='pro'
        svo_5.append(svo_q5)
    elif i==3:
        
        svo_q5='ind'
        svo_5.append(svo_q5) 
    else:
        svo_q5='None'
        svo_5.append(svo_q5)
        
#add the svo_5 col
raw_onlinefb['svo_5']=svo_5


#pro-I-Com
#svo_q6
svo_6=[]
for i in raw_onlinefb['svo_6']:
    if i==1:
        svo_q6='pro'
        svo_6.append(svo_q6)
    elif i==2:
        
        svo_q6='ind'
        svo_6.append(svo_q6)
    elif i==3:
        
        svo_q6='Com'
        svo_6.append(svo_q6)
    else:
        svo_q6='None'
        svo_6.append(svo_q6)
        
#add the svo_6 col
raw_onlinefb['svo_6']=svo_6

#pro-Com-I
#svo_7
svo_7=[]
for i in raw_onlinefb['svo_7']:
    if i==1:
        svo_q7='pro'
        svo_7.append(svo_q7)
    elif i==2:
        svo_q7='Com'
        svo_7.append(svo_q7)
    elif i==3:
        
        svo_q7='ind'
        svo_7.append(svo_q7)
    else:
        svo_q7='None'
        svo_7.append(svo_q7)
        
#add the svo_7 col
raw_onlinefb['svo_7']=svo_7

#Com-I-pro
#svo_8
svo_8=[]
for i in raw_onlinefb['svo_8']:
    if i==1:
        svo_q8='Com'
        svo_8.append(svo_q8)
    elif i==2:
        svo_q8='ind'
        svo_8.append(svo_q8)
    elif i==3:
        
        svo_q8='pro'
        svo_8.append(svo_q8)
    else:
        svo_q8='None'
        svo_8.append(svo_q8)
#add the svo_8 col
raw_onlinefb['svo_8']=svo_8

#I-pro-Com
#svo_9
svo_9=[]
for i in raw_onlinefb['svo_9']:
    if i==1:
        svo_q9='ind'
        svo_9.append(svo_q9)
    elif i==2:
        svo_q9='pro'
        svo_9.append(svo_q9)
    elif i==3:
        
        svo_q9='Com'
        svo_9.append(svo_q9)
    else:
        svo_q9='None'
        svo_9.append(svo_q9)

#add the svo_9 col
raw_onlinefb['svo_9']=svo_9
#i need to convert the number of times a word is there
pro = ['pro']
ind = ['ind']
com = ['Com']
raw_onlinefb["prosocial"] = raw_onlinefb.isin(pro).sum(axis='columns')
raw_onlinefb["individualistic"] = raw_onlinefb.isin(ind).sum(axis='columns')
raw_onlinefb["competetive"] = raw_onlinefb.isin(com).sum(axis='columns')
#create a column to clarify fainall y hwre every one should be in the category 
# if someone has 6 or more times made a decision you choose that category of SVO for the person
conditions = [
    (raw_onlinefb["prosocial"] >= 6),
    (raw_onlinefb["individualistic"] >=6) , (raw_onlinefb["competetive"] >= 6)]
values = ["prosocial", "individualistic", "competetive"]
raw_onlinefb['SVO_Final'] = np.select(conditions, values)


# In[27]:


#to prepare data for analysis of BIS/BAS
#Q75_1	Q75_2	Q75_3	Q75_4	
#Q75_5	Q75_6	Q116_1	Q116_2	Q116_3	Q116_4	Q116_5	Q116_6
#Q117_1	Q117_2	Q117_3	Q117_4	Q117_5
#Q117_6	Q118_1	Q118_2	Q118_3	Q118_4	Q118_5	Q118_6
#raw_onlinefb.columns.to_list()

#Items 1, 3-21, 23-24 are reverse-scored.
#Items 2 and 22 are scored normally.
 
#4 Subscales: Behavioral Inhibition System (BIS), Behavioral Activation System (BAS) Drive, BAS Fun Seeking, BAS Reward Responsiveness
#BIS:  items 2, 8, 13, 16, 19, 22, 24
#BAS Drive:  items 3, 9, 12, 21 
#BAS Fun Seeking:  items 5, 10, 15, 20 
#BAS Reward Responsiveness:  items 4, 7, 14, 18, 23
bis_bas=['Q75_1','Q75_2','Q75_3','Q75_4','Q75_5','Q75_6','Q116_1',
         'Q116_2','Q116_3',
         'Q116_4','Q116_5','Q116_6',
         'Q117_1','Q117_2','Q117_3','Q117_4',
         'Q117_5','Q117_6','Q118_1','Q118_2','Q118_3','Q118_4','Q118_5',
         'Q118_6']
BIS_Total=[2, 8, 13, 16, 19, 22, 24]
BAS_total=[3, 4, 5, 7, 9, 10, 12, 14, 15, 18, 20, 21, 23]
BAS_drive=[3, 9, 12, 21 ]
BAS_FUN=[5, 10, 15, 20]
BAS_rewards=[4, 7, 14, 18, 23]
Filler=[1,6,11,17]
BIS_BAS_num=[]
#to find qs excist in all and also to create a groupe separately
BAS_group=[BAS_total,BAS_FUN,BAS_rewards]

for i in range(1,25):

    if i in (BAS_total and BAS_FUN):
        i=str(i)+"_"+'total_fun'
        BIS_BAS_num.append(i)
    elif i in (BAS_total and BAS_rewards):
        i=str(i)+"_"+'total_reward'
        BIS_BAS_num.append(i)    

    elif i in BAS_drive:
        i=str(i)+"_"+'BAS_drive'
        BIS_BAS_num.append(i)
    elif i in BAS_FUN:
        i=str(i)+"_"+'BAS_FUN'
        BIS_BAS_num.append(i)    
    elif i in BAS_rewards:
        i=str(i)+"_"+'BAS_rewards'
        BIS_BAS_num.append(i)
    elif i in BIS_Total:
        i=str(i)+"_"+'BIS_Total'
        BIS_BAS_num.append(i)
    elif i in Filler:
        i=str(i)+"_"+'Filler'
        BIS_BAS_num.append(i)      

#make a dic to re_label the raw col names from raw data for BIS/BAS test
#bisbas_dic=dict(zip(bis_bas,BIS_BAS_num))   
raw_onlinefb=raw_onlinefb.rename(columns={'Q75_1': '1_Filler','Q75_2': '2_BIS_Total','Q75_3': '3_BAS_drive',
                                          'Q75_4': '4_total_reward','Q75_5': '5_total_fun','Q75_6': '6_Filler',
 'Q116_1': '7_total_reward', 'Q116_2': '8_BIS_Total', 'Q116_3': '9_BAS_drive', 'Q116_4': '10_total_fun', 'Q116_5': '11_Filler', 'Q116_6': '12_BAS_drive', 'Q117_1': '13_BIS_Total', 'Q117_2': '14_total_reward',
 'Q117_3': '15_total_fun', 'Q117_4': '16_BIS_Total', 'Q117_5': '17_Filler', 'Q117_6': '18_total_reward', 'Q118_1': '19_BIS_Total',
 'Q118_2': '20_total_fun', 'Q118_3': '21_BAS_drive', 'Q118_4': '22_BIS_Total', 'Q118_5': '23_total_reward', 'Q118_6': '24_BIS_Total'})

#there are some reverse scoring
raw_onlinefb['2_BIS_Total'] = raw_onlinefb['2_BIS_Total'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['22_BIS_Total'] = raw_onlinefb['22_BIS_Total'].map({1:4, 2:3, 3:2, 4:1})    

#adding columns based on total BIS or BAs and also sub-factors
BIS_Total_qs=['2_BIS_Total','8_BIS_Total','13_BIS_Total','16_BIS_Total','19_BIS_Total','22_BIS_Total','24_BIS_Total']
raw_onlinefb['BIS_Total_qs']=raw_onlinefb[BIS_Total_qs].sum(axis=1)

BAS_total_qs=['3_BAS_drive','4_total_reward','5_total_fun','7_total_reward','9_BAS_drive','10_total_fun',
           '12_BAS_drive','14_total_reward','15_total_fun','18_total_reward','20_total_fun','21_BAS_drive','23_total_reward']
raw_onlinefb['BAS_total_qs']=raw_onlinefb[BAS_total_qs].sum(axis=1)

BAS_drive=['3_BAS_drive','9_BAS_drive','12_BAS_drive','21_BAS_drive']
raw_onlinefb['BAS_drive']=raw_onlinefb[BAS_drive].sum(axis=1)

BAS_FUN=['5_total_fun','10_total_fun','15_total_fun','20_total_fun']
raw_onlinefb['BAS_FUN']=raw_onlinefb[BAS_FUN].sum(axis=1)

BAS_rewards=['4_total_reward','7_total_reward','14_total_reward','18_total_reward','23_total_reward']
raw_onlinefb['BAS_rewards']=raw_onlinefb[BAS_rewards].sum(axis=1)

Filler=['1_Filler','6_Filler','11_Filler','17_Filler']
raw_onlinefb['Filler']=raw_onlinefb[Filler].sum(axis=1)


# In[28]:


#to prepare BIS_15 qs for data analysis
#cols that include the bis 15 qs
Bis_15_raw=['Q82_1','Q82_2','Q82_3','Q82_4','Q82_5','Q85_1','Q85_2','Q85_3','Q85_4','Q85_5','Q86_1',
'Q86_2','Q86_3','Q86_4','Q86_5']
#non-planning impulsiveness
NPI=[1,5,7,8,15]
#motor impulsivity
MI=[2,10,12,13,9]
#Attention-based impulsivity.
ABI=[14,6,4,3,11]
#make a list of BIS qs
BIS_15_lis=[]
for i in range(1,16):
    if i in NPI:
        i=str(i)+"-"+'BIS-15-NPI'
        BIS_15_lis.append(i)
    if i in MI:
        i=str(i)+"-"+'BIS-15-MI'
        BIS_15_lis.append(i)
    if i in ABI:
        i=str(i)+"-"+'BIS-15-ABI'
        BIS_15_lis.append(i)

#need to synch raw with edited col names
#bis15_dic=dict(zip(Bis_15_raw,BIS_15_lis))  
#rename the bis15 cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q82_1': '1-BIS-15-NPI',
 'Q82_2': '2-BIS-15-MI',
 'Q82_3': '3-BIS-15-ABI', 'Q82_4': '4-BIS-15-ABI', 'Q82_5': '5-BIS-15-NPI', 'Q85_1': '6-BIS-15-ABI', 'Q85_2': '7-BIS-15-NPI', 'Q85_3': '8-BIS-15-NPI','Q85_4': '9-BIS-15-MI',
'Q85_5': '10-BIS-15-MI', 'Q86_1': '11-BIS-15-ABI',
 'Q86_2': '12-BIS-15-MI', 'Q86_3': '13-BIS-15-MI', 'Q86_4': '14-BIS-15-ABI', 'Q86_5': '15-BIS-15-NPI'})

#there are qs need to be reversed for scoring
#questions should be reversed 13,15,1,7,4
raw_onlinefb['13-BIS-15-MI'] = raw_onlinefb['13-BIS-15-MI'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['15-BIS-15-NPI']= raw_onlinefb['15-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})    
raw_onlinefb['1-BIS-15-NPI'] = raw_onlinefb['1-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['7-BIS-15-NPI'] = raw_onlinefb['7-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})   
raw_onlinefb['4-BIS-15-ABI'] = raw_onlinefb['4-BIS-15-ABI'].map({1:4, 2:3, 3:2, 4:1})

#make new cols for each subcategory of BIS15 and also a total one
bis_15_mi=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI']
raw_onlinefb['BIS_15_MI']=raw_onlinefb[bis_15_mi].sum(axis=1)

bis_15_npi=['1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI']
raw_onlinefb['BIS_15_NPI']=raw_onlinefb[bis_15_npi].sum(axis=1)

bis_15_abi=['3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
raw_onlinefb['BIS_15_ABI']=raw_onlinefb[bis_15_abi].sum(axis=1)

#create a col form all 15 items of BiS-15
BIS_15_total=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI',
       '1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI',
       '3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
raw_onlinefb['BIS_15_total']=raw_onlinefb[BIS_15_total].sum(axis=1)


# In[29]:


#BFI data prepration
BFI_Raw=['Q135_1', 'Q135_2', 'Q135_3', 'Q135_4', 'Q135_5', 'Q136_1',
         'Q136_2','Q136_3','Q136_4','Q136_5']
#five dimension of BFI
N=[4,9]
E=[1,6]
O=[5,10]
A=[2,7]
C=[3,8]
#list of BFI
BFI_list=[]
for i in range(1,11):
    if i in N:
        i=str(i)+"-"+'BFI_N'
        BFI_list.append(i)
    if i in E:
        i=str(i)+"-"+'BFI_E'
        BFI_list.append(i)
    if i in O:
        i=str(i)+"-"+'BFI_O'
        BFI_list.append(i)        
    if i in A:
        i=str(i)+"-"+'BFI_A'
        BFI_list.append(i)
    if i in C:
        i=str(i)+"-"+'BFI_C'
        BFI_list.append(i)

#need to synch raw with edited col names
BFI_dic=dict(zip(BFI_Raw,BFI_list))  

#rename the bis15 cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q135_1': '1-BFI_E', 'Q135_2': '2-BFI_A','Q135_3': '3-BFI_C',                                           'Q135_4': '4-BFI_N',
'Q135_5': '5-BFI_O','Q136_1': '6-BFI_E', 'Q136_2': '7-BFI_A', 'Q136_3': '8-BFI_C','Q136_4': '9-BFI_N', 'Q136_5': '10-BFI_O'})

#BFI- questions should be reversed 1,3,4,5,7
raw_onlinefb['1-BFI_E'] = raw_onlinefb['1-BFI_E'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['3-BFI_C'] = raw_onlinefb['3-BFI_C'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['4-BFI_N'] = raw_onlinefb['4-BFI_N'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['5-BFI_O'] = raw_onlinefb['5-BFI_O'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['7-BFI_A'] = raw_onlinefb['7-BFI_A'].map({1:5, 2:4, 3:3, 4:2,5:1})
#to calculate mean of each dimension
# N=[4,9]
# E=[1,6]
# O=[5,10]
# A=[2,7]
# C=[3,8]
#create new cols for calclualtion of each dim of BFI (mean)
bfi_n=['4-BFI_N','9-BFI_N']
raw_onlinefb['BFI_N']=raw_onlinefb[bfi_n].mean(axis=1)
bfi_e=['1-BFI_E','6-BFI_E']
raw_onlinefb['BFI_E']=raw_onlinefb[bfi_e].mean(axis=1)
bfi_o=['5-BFI_O','10-BFI_O']
raw_onlinefb['BFI_O']=raw_onlinefb[bfi_o].mean(axis=1)
bfi_a=['2-BFI_A','7-BFI_A']
raw_onlinefb['BFI_A']=raw_onlinefb[bfi_a].mean(axis=1)
bfi_c=['3-BFI_C','8-BFI_C']
raw_onlinefb['BFI_C']=raw_onlinefb[bfi_c].mean(axis=1)


# In[30]:



#CRT-there are three questions with right answer 
crt_ball=5
crt_machine=5
crt_lake=47
#raw col names
crt_raw=['Q88_1','Q89_1','Q90_1']
#rename the raw cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q88_1':'CRT_ball','Q89_1':'CRT_machine','Q90_1':'CRT_lake'})
#create empty list for three crts
crt_ball=[]
crt_machine=[]
crt_lake=[]
#there are different kind of answers for each crt in raw data
crt_ball_answers=['5','0,05','5 cent']
crt_machine_answers=['5','5 Minuten']
crt_lake_answers=['47','47 Tage']
for i in raw_onlinefb.CRT_ball:
    if i in crt_ball_answers:
        i='True'
        crt_ball.append(i)
    else:
        i='False'
        crt_ball.append(i)
for i in raw_onlinefb.CRT_machine:
    if i in crt_machine_answers:
        i='True'
        crt_machine.append(i)
    else:
        i='False'
        crt_machine.append(i)
for i in raw_onlinefb.CRT_machine:
    if i in crt_machine_answers:
        i='True'
        crt_machine.append(i)
    else:
        i='False'
        crt_machine.append(i)
for i in raw_onlinefb.CRT_lake:
    if i in crt_lake_answers:
        i='True'
        crt_lake.append(i)
    else:
        i='False'
        crt_lake.append(i)
#creating new graphs with results of for loop for each crt question
raw_onlinefb['crt_ball']=crt_ball
raw_onlinefb['crt_machine']=crt_lake
raw_onlinefb['crt_lake']=crt_lake


# In[31]:


#to prepare the STAI qs for data analysis
#For STAI, there are two set of qs, each 20.

#the name of stai cols in raw data
stais_raw=['Q67_1', 'Q67_2', 'Q67_3', 'Q67_4', 'Q67_5', 'Q119_1', 'Q119_2',
 'Q119_3', 'Q119_4', 'Q119_5', 'Q120_1', 'Q120_2', 'Q120_3', 'Q120_4', 'Q120_5',
 'Q121_1', 'Q121_2', 'Q121_3', 'Q121_4', 'Q121_5', 'Q71_1', 'Q71_2', 'Q71_3',
 'Q71_4', 'Q71_5', 'Q122_1', 'Q122_2', 'Q122_3', 'Q122_4', 'Q122_5', 'Q123_1', 'Q123_2',
 'Q123_3', 'Q123_4', 'Q123_5', 'Q124_1', 'Q124_2', 'Q124_3', 'Q124_4', 'Q124_5']
nums=[]
for i in range(1,41):
    i=str(i)+"-"+'stai'
    nums.append(i)
    
#need to synch raw with nums
stai_dic=dict(zip(stais_raw,nums)) 
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q67_1': '1-stai', 'Q67_2': '2-stai', 'Q67_3': '3-stai', 'Q67_4': '4-stai', 'Q67_5': '5-stai',
 'Q119_1': '6-stai', 'Q119_2': '7-stai', 'Q119_3': '8-stai', 'Q119_4': '9-stai', 'Q119_5': '10-stai', 'Q120_1': '11-stai', 'Q120_2': '12-stai',
 'Q120_3': '13-stai', 'Q120_4': '14-stai', 'Q120_5': '15-stai', 'Q121_1': '16-stai', 'Q121_2': '17-stai', 'Q121_3': '18-stai',
 'Q121_4': '19-stai', 'Q121_5': '20-stai', 'Q71_1': '21-stai', 'Q71_2': '22-stai', 'Q71_3': '23-stai', 'Q71_4': '24-stai',
                        'Q71_5': '25-stai', 'Q122_1': '26-stai','Q122_2': '27-stai', 'Q122_3': '28-stai', 'Q122_4': '29-stai', 'Q122_5': '30-stai',
 'Q123_1': '31-stai', 'Q123_2': '32-stai', 'Q123_3': '33-stai', 'Q123_4': '34-stai', 'Q123_5': '35-stai', 'Q124_1': '36-stai',
 'Q124_2': '37-stai', 'Q124_3': '38-stai', 'Q124_4': '39-stai', 'Q124_5': '40-stai'})
#to reverse the specific questions
##There are many questions reverted: Y1=[1,2,5,8,10,11,15,16,19,20] for Y2=[21,23,26,27,30,33,34,36,39]
reversed_qs=['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']
#reverse the answers
raw_onlinefb[['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']] =raw_onlinefb[['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']].replace({1:4, 2:3, 3:2, 4:1})
#calculate score of each participant for STAI
raw_onlinefb['stai_total']=raw_onlinefb[reversed_qs].sum(axis=1)


# In[32]:


#to prepare raw data from SDS 17
raw_sds=['Q33', 'Q65', 'Q49', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64']
# new col names
sds_num=[]
for i in range(1,18):
    i=str(i)+"-"+'SDS17'
    sds_num.append(i)
#need to synch raw with nums
sds17_dic=dict(zip(raw_sds,sds_num)) 
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q33': '1-SDS17', 'Q65': '2-SDS17', 'Q49': '3-SDS17', 'Q51': '4-SDS17',
                                          'Q52': '5-SDS17', 'Q53': '6-SDS17', 'Q54': '7-SDS17', 'Q55': '8-SDS17', 
                                          'Q56': '9-SDS17', 'Q57': '10-SDS17',
 'Q58': '11-SDS17', 'Q59': '12-SDS17', 'Q60': '13-SDS17','Q61': '14-SDS17', 
 'Q62': '15-SDS17', 'Q63': '16-SDS17', 'Q64': '17-SDS17'})
#there is no reverse scoring in SDS17 but i should only have the sum of 1 in raw data which means True
sds_cols=['1-SDS17', '2-SDS17', '3-SDS17', '4-SDS17', '5-SDS17', '6-SDS17', '7-SDS17', '8-SDS17', '9-SDS17', '10-SDS17',
 '11-SDS17', '12-SDS17', '13-SDS17', '14-SDS17', '15-SDS17', '16-SDS17','17-SDS17']

raw_onlinefb['sds_total']=raw_onlinefb[raw_onlinefb[sds_cols]==1].sum(axis=1)         


# In[33]:


#to prepare raw data of MEQ
# MEQ scoring:
# Q1->1-6
# Q2->4-1
# Q3->1-5
# Q4->1-5
# Q5->1-4
meq_ls=['Q12', 'Q13', 'Q14', 'Q15', 'Q16']
#change the name of cols
meq_cols=['meq-1','meq-2','meq-3','meq-4','meq-5']
#need to synch raw with nums
meq_dict=dict(zip(meq_ls,meq_cols))
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q12': 'meq-1', 'Q13': 'meq-2', 'Q14': 'meq-3', 'Q15': 'meq-4','Q16':'meq-5'})
#teh second q should be reversed
raw_onlinefb['meq-2'] = raw_onlinefb['meq-2'].map({4:1, 3:2, 2:3, 1:4})
raw_onlinefb['meq_total']=raw_onlinefb[['meq-1','meq-2','meq-3','meq-4','meq-5']].sum(axis=1)
#the higher the number more ecening person

#to prepare escale
raw_cols=['Q138_1', 'Q138_2', 'Q138_3', 'Q138_4', 'Q138_5', 'Q140_1', 'Q140_2', 'Q140_3', 'Q140_4','Q140_5', 'Q141_1',
 'Q141_2', 'Q141_3', 'Q141_4', 'Q141_5', 'Q142_1', 'Q142_2', 'Q142_3', 'Q142_4', 'Q142_5', 'Q143_1', 'Q143_2','Q143_3', 
          'Q143_4', 'Q143_5']
# Factor 1- cognitive-sensitivity [CS] 14,17,19,24,21
# Factor 2- emotional-sensitivity [ES]1,2, 5, 7, 9, 11
# Factor 3- emotional-concern [EC]10, 15, 18, 20 (negatively poled), 22, 23, 25 
# Factor 4- cognitive-concern[CC] 3, 4, 6, 8, 13
cs=[14,17,19,24,21]
es=[1,2, 5, 7, 9, 11]
ec=[10, 15, 18, 20, 22, 23, 25 ]
cc=[3, 4, 6, 8, 13]
e_scale=[]
for i in range(1,26):
    if i in cs:
        i=str(i)+"-"+'escale_cs'
        e_scale.append(i)
    if i in es:
        i=str(i)+"-"+'escale_es'
        e_scale.append(i)
    if i in ec:
        i=str(i)+"-"+'escale_ec'
        e_scale.append(i)
    if i in cc:
        i=str(i)+"-"+'escale_cc'
        e_scale.append(i)
        
escale_dict=dict(zip(raw_cols,e_scale))        
raw_onlinefb=raw_onlinefb.rename(columns={'Q138_1': '1-escale_es', 'Q138_2': '2-escale_es', 
                                          'Q138_3': '3-escale_cc', 'Q138_4': '4-escale_cc', 'Q138_5': '5-escale_es', 'Q140_1': '6-escale_cc',
 'Q140_2': '7-escale_es', 'Q140_3': '8-escale_cc','Q140_4': '9-escale_es', 'Q140_5': '10-escale_ec', 
                                          'Q141_1': '11-escale_es', 'Q141_2': '13-escale_cc',
 'Q141_3': '14-escale_cs', 'Q141_4': '15-escale_ec', 'Q141_5': '17-escale_cs', 'Q142_1': '18-escale_ec',
                                          'Q142_2': '19-escale_cs', 'Q142_3': '20-escale_ec',
 'Q142_4': '21-escale_cs', 'Q142_5': '22-escale_ec', 'Q143_1': '23-escale_ec', 'Q143_2': '24-escale_cs',
                                          'Q143_3': '25-escale_ec'})
raw_onlinefb['20-escale_ec'] = raw_onlinefb['20-escale_ec'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['e-scale_total']=raw_onlinefb[e_scale].sum(axis=1)


# In[34]:


#to remove the empty cols of avatar test and bringing all under each other
# i need to add 2 to empty cells in col Q500
# then only for rows of q500 that are 2 and if 1_Q194_First Click is empty 
#remove the cols from 1_Q194_First Click to 1_Q196_First Click

#add 2 to Q500
raw_onlinefb.Q500.fillna(2,inplace=True)
 

# Replacing empty string with np.NaN
import numpy as np
raw_onlinefb = raw_onlinefb.replace('', np.nan)
df_left=raw_onlinefb[(~raw_onlinefb['1_Q194_First Click'].isnull())&(raw_onlinefb['Q500']==2)]
#remove the unwanted cols from df_left

df_left.drop(columns=df_left.loc[:,'1_Q196_First Click':'64_Q193'], axis=1,inplace=True)
#creating two separated dfs to get rid of duplicate cols
df_right=raw_onlinefb[(raw_onlinefb['1_Q194_First Click'].isnull())&(raw_onlinefb['Q500']==2)]

#remove the unwanted cols from df_right
df_right.drop(columns=df_right.loc[:,'Q500_1_TEXT':'64_Q175'], axis=1,inplace=True)

#df_left name of cols 
col_df_left=df_left.loc[:,'1_Q194_First Click':'64_Q175'].columns.to_list()
#df_right name of cols
col_df_right=df_right.loc[:,'1_Q196_First Click':'64_Q193'].columns.to_list()
#make dict for both cols names
dict_col_name=dict(zip(col_df_right,col_df_left))  
#rename the df_right specific col names by df_left col names
df_right=df_right.rename(dict_col_name,axis=1)
#add the information which order of perspective they did: left or right
df_left['perspective-order']='left'
df_right['perspective-order']='right'
#encode the right and wrong questions
#for now I coded not answered 'NONE'qs as wrong answered question
#left order 5=stimmt nicht überein 4=stimmt überein 
#right order 4=stimmt nicht überein , 5=stimmt überein 
#questions and correct answers

#question_left_5 as correct answer
ql5c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_left[ql5c]= np.where(df_left[ql5c]==5, 0, 1)
#question_right_4 as correct answer
qr4c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_right[qr4c]= np.where(df_right[qr4c]==4, 0, 1)
#question_left_4 as correct answer
ql4c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_left[ql4c]= np.where(df_left[ql4c]==4, 0, 1)
#question_right_5 as correct answer
qr5c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_right[qr5c]= np.where(df_right[qr5c]==5, 0, 1)
# i need to create four groups 1. self congurent (sc) 2. self incongruent(si) 3.other congruent(oc) 4.other incongruent (oi)
sc=['10_Q175','12_Q175','15_Q175','16_Q175','20_Q175','25_Q175','26_Q175','27_Q175','28_Q175','33_Q175','34_Q175','42_Q175','47_Q175','49_Q175'
,'61_Q175','62_Q175']
si=['1_Q175','7_Q175','14_Q175','18_Q175','21_Q175','31_Q175','36_Q175','38_Q175','41_Q175','46_Q175','51_Q175','56_Q175','59_Q175','60_Q175'
,'63_Q175','64_Q175']
oc=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','11_Q175','17_Q175','24_Q175','29_Q175','32_Q175','37_Q175','44_Q175','45_Q175','50_Q175'
,'54_Q175','58_Q175']
oi=['5_Q175','8_Q175','13_Q175','19_Q175','22_Q175','23_Q175','30_Q175','35_Q175','39_Q175','40_Q175','43_Q175','48_Q175','52_Q175','53_Q175'
,'55_Q175','57_Q175']
# remove trials 3 and 63 due to wrong link of picture for participants before the date 17.10.2022
#append both dfs
df=df_left.append(df_right)
#column for sum of sc
df['self_congurent']= df.apply(lambda row: row[sc].sum(),axis=1)
df['self_congurent']=(16-df['self_congurent'])/16
#column for sum of si
df['self_incongurent']=df.apply(lambda row: row[si].sum(),axis=1)
df['self_incongurent']=(16-df['self_incongurent'])/16
#column for sum of oc
df['other_congurent']=df.apply(lambda row: row[oc].sum(),axis=1)
df['other_congurent']=(16-df['other_congurent'])/16
#column for sum of oi
df['other_incongurent']=df.apply(lambda row: row[oi].sum(),axis=1)
df['other_incongurent']=(16-df['other_incongurent'])/16
#calculate the incongurent! if value is >0 means self incongurent and if <0 other ingurent and 0 means no ingurennt
df['perspective']=(df['self_incongurent']-df['self_congurent'])-(df['other_incongurent']-df['other_congurent'])


# In[35]:


df.rename(columns={'StartDate':'Date','Q132_1':'Vp-Code'},inplace=True)


# In[36]:


#export only necessary cols for statistical analysis
cols_wanted=['Date','Duration (in seconds)','Vp-Code','TICS', 'TICS_control1', 'TICS_control2', 'TICS_control3', 'SVO_Final','BIS_Total_qs', 'BAS_total_qs', 'BAS_drive', 'BAS_FUN', 'BAS_rewards', 'BIS_15_MI', 'BIS_15_NPI', 'BIS_15_ABI',
 'BIS_15_total', 'BFI_N', 'BFI_E', 'BFI_O', 'BFI_A', 'BFI_C', 'crt_ball', 'crt_machine', 'crt_lake', 'stai_total', 'sds_total',
 'meq_total', 'e-scale_total','perspective']


# In[37]:


df=df[cols_wanted]
#i need to change the name of ANPE07B which is for the VP HETO16B
df=df.sort_values(by='Vp-Code')
df.iloc[2,2]='HETO16B'


# In[38]:


#export the edited DF
df_append=pd.ExcelWriter('df_online_DOM.xlsx')
df.to_excel(df_append)
df_append.save()




/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-35-39a247cfb2a9>:812: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left['perspective-orde

In [36]:
#Online Luca


#here I will export the output of online QS of lucas' study
#exported data should be numeric
import pandas as pd
import seaborn as sns
import numpy as np
#upload raw data from inline questionnaire
raw_onlinefb=pd.read_excel("online_luca_raw.xlsx")
#drop the first unwanted row
raw_onlinefb=raw_onlinefb.drop([0], axis=0)

#drop unwanted columns
raw_onlinefb=raw_onlinefb.drop([ 'EndDate', 'Status', 'IPAddress','Finished', 
       'RecipientLastName', 'RecordedDate', 'ResponseId', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude',
                                'LocationLongitude', 'UserLanguage'], axis=1)
distribution=['anonymous','qr']
raw_onlinefb=raw_onlinefb.loc[raw_onlinefb['DistributionChannel'].isin(distribution)]
#drop the participants did not complete their survey
raw_onlinefb=raw_onlinefb[raw_onlinefb['Progress']==100] 


#drop the cols of VP codes if they are empty
raw_onlinefb.dropna(subset=['Q132_1'], inplace=True)

#columns that cover TICS QS
df_tics=['Q2_1', 'Q2_2', 'Q2_3', 'Q2_4', 'Q2_5', 'Q111_1', 'Q111_2', 'Q111_3', 'Q111_4', 'Q111_5',
         'Q76_1', 'Q76_2','Q76_3', 'Q76_5', 'Q110_1', 'Q110_2', 'Q110_3', 'Q110_4', 
         'Q110_5', 'Q77_1', 'Q77_2', 'Q77_3', 'Q77_4', 'Q77_5', 'Q112_1',  
         'Q112_2', 'Q112_3', 'Q112_4', 'Q112_5', 'Q78_1', 'Q78_2','Q78_3', 
         'Q78_4', 'Q78_5', 'Q113_1', 'Q113_2', 'Q113_3', 'Q113_4',
         'Q113_5', 'Q79_1', 'Q79_2', 'Q79_3', 'Q79_4', 'Q114_1','Q114_2', 
         'Q114_3', 'Q114_4', 'Q114_5', 'Q80_1',  'Q80_3',
         'Q80_4', 'Q80_5', 'Q115_1', 'Q115_2', 'Q115_3', 'Q115_4',
         'Q115_5']

#There are three control question that 
#every VP need to have three of that in his/her DF
control_qs=['Q76_4', 'Q79_5','Q80_2',]
Control_1=[]
Control_2=[]
Control_3=[]
for i in raw_onlinefb[control_qs[0]]:
    if i==4:
        a='True'
        Control_1.append(a)
    else:
        
        b='false'
        Control_1.append(b)
    
for i in raw_onlinefb[control_qs[1]]:
    if i==1:
        a='True'
        Control_2.append(a)
    else:
        
        b='False'
        Control_2.append(b)   
for i in raw_onlinefb[control_qs[2]]:
    if i==2:
        a='True'
        Control_3.append(a)
    else:
        
        b='False'
        Control_3.append(b)    

#identifying the 9-subgroups of TICS
#category and its questions
#items covering work overload
WOL=[1,4,44,54,17,27,38,50]
#items covering social overload
SO=[49,57,7,19,28,39]
#items covering Pressure to Perform
PP=[8,12,14,22,30,23,32,40,43]
#items covering work discontent
WD=[5,10,13,41,21,37,48,53]
#items covering Excessive Demands at Work
EDW=[3,47,20,24,35,55]
#items covering Lack of Social Recognition
LSR=[2,18,31,46]
#items covering Social Tensions
ST=[6,33,15,26,45,52]
#items covering social isolation
SI=[29,34,11,42,51,56]
#chronic worring
CW=[9,16,25,36]
tics_num=[]
for i in range(1,61):
    n=i
    if n in LSR:
        n=str(n)+"_"+'LSR'
        tics_num.append(n)
    elif n in SO:
        n=str(n)+"_"+'SO'
        tics_num.append(n)
    elif n in PP:
        n=str(n)+"_"+'PP'
        tics_num.append(n)
    elif n in WD:
        n=str(n)+"_"+'WD'
        tics_num.append(n)
    elif n in EDW:
        n=str(n)+"_"+'EDW'
        tics_num.append(n)
    elif n in WOL:
        n=str(n)+"_"+'WOL'
        tics_num.append(n)
    elif n in ST:
        n=str(n)+"_"+'ST'
        tics_num.append(n)
    elif n in SI:
        n=str(n)+"_"+'SI'
        tics_num.append(n)
    elif n in CW:
        n=str(n)+"_"+'CW'
        tics_num.append(n)
        
#Convert label of tic cols into numbers
#with shoeing they belong to which subcategory of tics
tics_dic=dict(zip(df_tics,tics_num))   
raw_onlinefb=raw_onlinefb.rename(columns={'Q2_1': '1_WOL', 'Q2_2': '2_LSR' ,'Q2_3': '3_EDW', 'Q2_4': '4_WOL',
                                          'Q2_5': '5_WD',
 'Q111_1': '6_ST', 'Q111_2': '7_SO', 'Q111_3': '8_PP', 'Q111_4': '9_CW', 'Q111_5': '10_WD', 'Q76_1': '11_SI', 'Q76_2': '12_PP', 'Q76_3': '13_WD',
 'Q76_5': '14_PP',
 'Q110_1': '15_ST', 'Q110_2': '16_CW', 'Q110_3': '17_WOL', 'Q110_4': '18_LSR', 'Q110_5': '19_SO', 'Q77_1': '20_EDW', 'Q77_2': '21_WD', 'Q77_3': '22_PP', 'Q77_4': '23_PP', 'Q77_5': '24_EDW', 'Q112_1': '25_CW', 'Q112_2': '26_ST', 'Q112_3': '27_WOL', 'Q112_4': '28_SO', 'Q112_5': '29_SI', 'Q78_1': '30_PP',
 'Q78_2': '31_LSR',
 'Q78_3': '32_PP',
 'Q78_4': '33_ST', 'Q78_5': '34_SI', 'Q113_1': '35_EDW', 'Q113_2': '36_CW', 'Q113_3': '37_WD', 'Q113_4': '38_WOL', 'Q113_5': '39_SO', 'Q79_1': '40_PP', 'Q79_2': '41_WD',
 'Q79_3': '42_SI', 'Q79_4': '43_PP', 'Q114_1': '44_WOL', 'Q114_2': '45_ST', 'Q114_3': '46_LSR', 'Q114_4': '47_EDW', 'Q114_5': '48_WD',
 'Q80_1': '49_SO', 'Q80_3': '50_WOL', 'Q80_4': '51_SI', 'Q80_5': '52_ST',
 'Q115_1': '53_WD', 'Q115_2': '54_WOL', 'Q115_3': '55_EDW', 'Q115_4': '56_SI', 'Q115_5': '57_SO'})
raw_onlinefb['TICS']=raw_onlinefb[tics_num].sum(axis=1)
raw_onlinefb['TICS_control1']=Control_1
raw_onlinefb['TICS_control2']=Control_2
raw_onlinefb['TICS_control3']=Control_3


#SVO data preparation
#to calculate the social value orientation
svo=['Q92_1','Q103_1','Q102_1','Q101_1','Q100_1','Q99_1','Q98_1','Q97_1','Q96_1',]
svo_num=['svo_1','svo_2','svo_3','svo_4','svo_5','svo_6','svo_7','svo_8','svo_9']
svo_dict=dict(zip(svo,svo_num))
raw_onlinefb=raw_onlinefb.rename(columns={'Q92_1': 'svo_1',
 'Q103_1': 'svo_2',
 'Q102_1': 'svo_3',
 'Q101_1': 'svo_4',
 'Q100_1': 'svo_5',
 'Q99_1': 'svo_6',
 'Q98_1': 'svo_7',
 'Q97_1': 'svo_8',
 'Q96_1': 'svo_9'})
#individualistic is H
#Competitive Com
# properative pro
#the order of orientations in online qs of SVO
#I-Com-pro	Com-pro-I	pro-I-Com	I-Com-pro	Com-pro-I
#pro-I-Com	pro-Com-I	Com-I-pro	I-pro-Com
#0 means pro/1 means Com/ 2 means I
#I-Com-pro
#the empty rows are called NONE
svo_1=[]
for i in raw_onlinefb['svo_1']:
    if i==1:
        svo_q1='ind'
        svo_1.append(svo_q1)
    elif i==2:
        
        svo_q1='Com'
        svo_1.append(svo_q1)
    elif i==3:
        
        svo_q1='pro'
        svo_1.append(svo_q1)  
    else:
        svo_q1='None'
        svo_1.append(svo_q1)
        
#add the svo_1 col
raw_onlinefb['svo_1']=svo_1
#Com-pro-I       
#svo_q2
svo_2=[]
for i in raw_onlinefb['svo_2']:
    if i==1:
        svo_q2='Com'
        svo_2.append(svo_q2)
    elif i==2:
        
        svo_q2='pro'
        svo_2.append(svo_q2)
    elif i==3:
        
        svo_q2='ind'
        svo_2.append(svo_q2) 
    else:
        svo_q2='None'
        svo_2.append(svo_q2)
        
    
        
#add the svo_2 col
raw_onlinefb['svo_2']=svo_2
#pro-I-Com
#svo_q3
svo_3=[]
for i in raw_onlinefb['svo_3']:
    if i==1:
        svo_q3='pro'
        svo_3.append(svo_q3)
    elif i==2:
        
        svo_q3='ind'
        svo_3.append(svo_q3)
    elif i==3:
        
        svo_q3='Com'
        svo_3.append(svo_q3)
    else:
        svo_q3='None'
        svo_3.append(svo_q3)
        
#add the svo_3 col
raw_onlinefb['svo_3']=svo_3

#I-Com-pro
svo_4=[]
for i in raw_onlinefb['svo_4']:
    if i==1:
        svo_q4='ind'
        svo_4.append(svo_q4)
    elif i==2:
        
        svo_q4='Com'
        svo_4.append(svo_q4)
    
    elif i==3:
        
        svo_q4='pro'
        svo_4.append(svo_q4) 
    else:
        svo_q4='None'
        svo_4.append(svo_q4)
        
#add the svo_4 col
raw_onlinefb['svo_4']=svo_4  


#Com-pro-I       
#svo_q5
svo_5=[]
for i in raw_onlinefb['svo_5']:
    if i==1:
        svo_q5='Com'
        svo_5.append(svo_q5)
    elif i==2:
        
        svo_q5='pro'
        svo_5.append(svo_q5)
    elif i==3:
        
        svo_q5='ind'
        svo_5.append(svo_q5) 
    else:
        svo_q5='None'
        svo_5.append(svo_q5)
        
#add the svo_5 col
raw_onlinefb['svo_5']=svo_5


#pro-I-Com
#svo_q6
svo_6=[]
for i in raw_onlinefb['svo_6']:
    if i==1:
        svo_q6='pro'
        svo_6.append(svo_q6)
    elif i==2:
        
        svo_q6='ind'
        svo_6.append(svo_q6)
    elif i==3:
        
        svo_q6='Com'
        svo_6.append(svo_q6)
    else:
        svo_q6='None'
        svo_6.append(svo_q6)
        
#add the svo_6 col
raw_onlinefb['svo_6']=svo_6

#pro-Com-I
#svo_7
svo_7=[]
for i in raw_onlinefb['svo_7']:
    if i==1:
        svo_q7='pro'
        svo_7.append(svo_q7)
    elif i==2:
        svo_q7='Com'
        svo_7.append(svo_q7)
    elif i==3:
        
        svo_q7='ind'
        svo_7.append(svo_q7)
    else:
        svo_q7='None'
        svo_7.append(svo_q7)
        
#add the svo_7 col
raw_onlinefb['svo_7']=svo_7

#Com-I-pro
#svo_8
svo_8=[]
for i in raw_onlinefb['svo_8']:
    if i==1:
        svo_q8='Com'
        svo_8.append(svo_q8)
    elif i==2:
        svo_q8='ind'
        svo_8.append(svo_q8)
    elif i==3:
        
        svo_q8='pro'
        svo_8.append(svo_q8)
    else:
        svo_q8='None'
        svo_8.append(svo_q8)
#add the svo_8 col
raw_onlinefb['svo_8']=svo_8

#I-pro-Com
#svo_9
svo_9=[]
for i in raw_onlinefb['svo_9']:
    if i==1:
        svo_q9='ind'
        svo_9.append(svo_q9)
    elif i==2:
        svo_q9='pro'
        svo_9.append(svo_q9)
    elif i==3:
        
        svo_q9='Com'
        svo_9.append(svo_q9)
    else:
        svo_q9='None'
        svo_9.append(svo_q9)

#add the svo_9 col
raw_onlinefb['svo_9']=svo_9
#i need to convert the number of times a word is there
pro = ['pro']
ind = ['ind']
com = ['Com']
raw_onlinefb["prosocial"] = raw_onlinefb.isin(pro).sum(axis='columns')
raw_onlinefb["individualistic"] = raw_onlinefb.isin(ind).sum(axis='columns')
raw_onlinefb["competetive"] = raw_onlinefb.isin(com).sum(axis='columns')
#create a column to clarify fainall y hwre every one should be in the category 
# if someone has 6 or more times made a decision you choose that category of SVO for the person
conditions = [
    (raw_onlinefb["prosocial"] >= 6),
    (raw_onlinefb["individualistic"] >=6) , (raw_onlinefb["competetive"] >= 6)]
values = ["prosocial", "individualistic", "competetive"]
raw_onlinefb['SVO_Final'] = np.select(conditions, values)


# In[3]:


#to prepare data for analysis of BIS/BAS
#Q75_1	Q75_2	Q75_3	Q75_4	
#Q75_5	Q75_6	Q116_1	Q116_2	Q116_3	Q116_4	Q116_5	Q116_6
#Q117_1	Q117_2	Q117_3	Q117_4	Q117_5
#Q117_6	Q118_1	Q118_2	Q118_3	Q118_4	Q118_5	Q118_6
#raw_onlinefb.columns.to_list()

#Items 1, 3-21, 23-24 are reverse-scored.
#Items 2 and 22 are scored normally.
 
#4 Subscales: Behavioral Inhibition System (BIS), Behavioral Activation System (BAS) Drive, BAS Fun Seeking, BAS Reward Responsiveness
#BIS:  items 2, 8, 13, 16, 19, 22, 24
#BAS Drive:  items 3, 9, 12, 21 
#BAS Fun Seeking:  items 5, 10, 15, 20 
#BAS Reward Responsiveness:  items 4, 7, 14, 18, 23
bis_bas=['Q75_1','Q75_2','Q75_3','Q75_4','Q75_5','Q75_6','Q116_1',
         'Q116_2','Q116_3',
         'Q116_4','Q116_5','Q116_6',
         'Q117_1','Q117_2','Q117_3','Q117_4',
         'Q117_5','Q117_6','Q118_1','Q118_2','Q118_3','Q118_4','Q118_5',
         'Q118_6']
BIS_Total=[2, 8, 13, 16, 19, 22, 24]
BAS_total=[3, 4, 5, 7, 9, 10, 12, 14, 15, 18, 20, 21, 23]
BAS_drive=[3, 9, 12, 21 ]
BAS_FUN=[5, 10, 15, 20]
BAS_rewards=[4, 7, 14, 18, 23]
Filler=[1,6,11,17]
BIS_BAS_num=[]
#to find qs excist in all and also to create a groupe separately
BAS_group=[BAS_total,BAS_FUN,BAS_rewards]

for i in range(1,25):

    if i in (BAS_total and BAS_FUN):
        i=str(i)+"_"+'total_fun'
        BIS_BAS_num.append(i)
    elif i in (BAS_total and BAS_rewards):
        i=str(i)+"_"+'total_reward'
        BIS_BAS_num.append(i)    

    elif i in BAS_drive:
        i=str(i)+"_"+'BAS_drive'
        BIS_BAS_num.append(i)
    elif i in BAS_FUN:
        i=str(i)+"_"+'BAS_FUN'
        BIS_BAS_num.append(i)    
    elif i in BAS_rewards:
        i=str(i)+"_"+'BAS_rewards'
        BIS_BAS_num.append(i)
    elif i in BIS_Total:
        i=str(i)+"_"+'BIS_Total'
        BIS_BAS_num.append(i)
    elif i in Filler:
        i=str(i)+"_"+'Filler'
        BIS_BAS_num.append(i)      

#make a dic to re_label the raw col names from raw data for BIS/BAS test
#bisbas_dic=dict(zip(bis_bas,BIS_BAS_num))   
raw_onlinefb=raw_onlinefb.rename(columns={'Q75_1': '1_Filler','Q75_2': '2_BIS_Total','Q75_3': '3_BAS_drive',
                                          'Q75_4': '4_total_reward','Q75_5': '5_total_fun','Q75_6': '6_Filler',
 'Q116_1': '7_total_reward', 'Q116_2': '8_BIS_Total', 'Q116_3': '9_BAS_drive', 'Q116_4': '10_total_fun', 'Q116_5': '11_Filler', 'Q116_6': '12_BAS_drive', 'Q117_1': '13_BIS_Total', 'Q117_2': '14_total_reward',
 'Q117_3': '15_total_fun', 'Q117_4': '16_BIS_Total', 'Q117_5': '17_Filler', 'Q117_6': '18_total_reward', 'Q118_1': '19_BIS_Total',
 'Q118_2': '20_total_fun', 'Q118_3': '21_BAS_drive', 'Q118_4': '22_BIS_Total', 'Q118_5': '23_total_reward', 'Q118_6': '24_BIS_Total'})

#there are some reverse scoring
raw_onlinefb['2_BIS_Total'] = raw_onlinefb['2_BIS_Total'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['22_BIS_Total'] = raw_onlinefb['22_BIS_Total'].map({1:4, 2:3, 3:2, 4:1})    

#adding columns based on total BIS or BAs and also sub-factors
BIS_Total_qs=['2_BIS_Total','8_BIS_Total','13_BIS_Total','16_BIS_Total','19_BIS_Total','22_BIS_Total','24_BIS_Total']
raw_onlinefb['BIS_Total_qs']=raw_onlinefb[BIS_Total_qs].sum(axis=1)

BAS_total_qs=['3_BAS_drive','4_total_reward','5_total_fun','7_total_reward','9_BAS_drive','10_total_fun',
           '12_BAS_drive','14_total_reward','15_total_fun','18_total_reward','20_total_fun','21_BAS_drive','23_total_reward']
raw_onlinefb['BAS_total_qs']=raw_onlinefb[BAS_total_qs].sum(axis=1)

BAS_drive=['3_BAS_drive','9_BAS_drive','12_BAS_drive','21_BAS_drive']
raw_onlinefb['BAS_drive']=raw_onlinefb[BAS_drive].sum(axis=1)

BAS_FUN=['5_total_fun','10_total_fun','15_total_fun','20_total_fun']
raw_onlinefb['BAS_FUN']=raw_onlinefb[BAS_FUN].sum(axis=1)

BAS_rewards=['4_total_reward','7_total_reward','14_total_reward','18_total_reward','23_total_reward']
raw_onlinefb['BAS_rewards']=raw_onlinefb[BAS_rewards].sum(axis=1)

Filler=['1_Filler','6_Filler','11_Filler','17_Filler']
raw_onlinefb['Filler']=raw_onlinefb[Filler].sum(axis=1)


# In[4]:


#to prepare BIS_15 qs for data analysis
#cols that include the bis 15 qs
Bis_15_raw=['Q82_1','Q82_2','Q82_3','Q82_4','Q82_5','Q85_1','Q85_2','Q85_3','Q85_4','Q85_5','Q86_1',
'Q86_2','Q86_3','Q86_4','Q86_5']
#non-planning impulsiveness
NPI=[1,5,7,8,15]
#motor impulsivity
MI=[2,10,12,13,9]
#Attention-based impulsivity.
ABI=[14,6,4,3,11]
#make a list of BIS qs
BIS_15_lis=[]
for i in range(1,16):
    if i in NPI:
        i=str(i)+"-"+'BIS-15-NPI'
        BIS_15_lis.append(i)
    if i in MI:
        i=str(i)+"-"+'BIS-15-MI'
        BIS_15_lis.append(i)
    if i in ABI:
        i=str(i)+"-"+'BIS-15-ABI'
        BIS_15_lis.append(i)

#need to synch raw with edited col names
#bis15_dic=dict(zip(Bis_15_raw,BIS_15_lis))  
#rename the bis15 cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q82_1': '1-BIS-15-NPI',
 'Q82_2': '2-BIS-15-MI',
 'Q82_3': '3-BIS-15-ABI', 'Q82_4': '4-BIS-15-ABI', 'Q82_5': '5-BIS-15-NPI', 'Q85_1': '6-BIS-15-ABI', 'Q85_2': '7-BIS-15-NPI', 'Q85_3': '8-BIS-15-NPI','Q85_4': '9-BIS-15-MI',
'Q85_5': '10-BIS-15-MI', 'Q86_1': '11-BIS-15-ABI',
 'Q86_2': '12-BIS-15-MI', 'Q86_3': '13-BIS-15-MI', 'Q86_4': '14-BIS-15-ABI', 'Q86_5': '15-BIS-15-NPI'})

#there are qs need to be reversed for scoring
#questions should be reversed 13,15,1,7,4
raw_onlinefb['13-BIS-15-MI'] = raw_onlinefb['13-BIS-15-MI'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['15-BIS-15-NPI']= raw_onlinefb['15-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})    
raw_onlinefb['1-BIS-15-NPI'] = raw_onlinefb['1-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})
raw_onlinefb['7-BIS-15-NPI'] = raw_onlinefb['7-BIS-15-NPI'].map({1:4, 2:3, 3:2, 4:1})   
raw_onlinefb['4-BIS-15-ABI'] = raw_onlinefb['4-BIS-15-ABI'].map({1:4, 2:3, 3:2, 4:1})

#make new cols for each subcategory of BIS15 and also a total one
bis_15_mi=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI']
raw_onlinefb['BIS_15_MI']=raw_onlinefb[bis_15_mi].sum(axis=1)

bis_15_npi=['1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI']
raw_onlinefb['BIS_15_NPI']=raw_onlinefb[bis_15_npi].sum(axis=1)

bis_15_abi=['3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
raw_onlinefb['BIS_15_ABI']=raw_onlinefb[bis_15_abi].sum(axis=1)

#create a col form all 15 items of BiS-15
BIS_15_total=['2-BIS-15-MI','9-BIS-15-MI','10-BIS-15-MI','12-BIS-15-MI','13-BIS-15-MI',
       '1-BIS-15-NPI','5-BIS-15-NPI','7-BIS-15-NPI','8-BIS-15-NPI','15-BIS-15-NPI',
       '3-BIS-15-ABI','4-BIS-15-ABI','6-BIS-15-ABI','11-BIS-15-ABI','14-BIS-15-ABI']
raw_onlinefb['BIS_15_total']=raw_onlinefb[BIS_15_total].sum(axis=1)


# In[5]:


#BFI data prepration
BFI_Raw=['Q135_1', 'Q135_2', 'Q135_3', 'Q135_4', 'Q135_5', 'Q136_1',
         'Q136_2','Q136_3','Q136_4','Q136_5']
#five dimension of BFI
N=[4,9]
E=[1,6]
O=[5,10]
A=[2,7]
C=[3,8]
#list of BFI
BFI_list=[]
for i in range(1,11):
    if i in N:
        i=str(i)+"-"+'BFI_N'
        BFI_list.append(i)
    if i in E:
        i=str(i)+"-"+'BFI_E'
        BFI_list.append(i)
    if i in O:
        i=str(i)+"-"+'BFI_O'
        BFI_list.append(i)        
    if i in A:
        i=str(i)+"-"+'BFI_A'
        BFI_list.append(i)
    if i in C:
        i=str(i)+"-"+'BFI_C'
        BFI_list.append(i)

#need to synch raw with edited col names
BFI_dic=dict(zip(BFI_Raw,BFI_list))  

#rename the bis15 cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q135_1': '1-BFI_E', 'Q135_2': '2-BFI_A','Q135_3': '3-BFI_C',                                           'Q135_4': '4-BFI_N',
'Q135_5': '5-BFI_O','Q136_1': '6-BFI_E', 'Q136_2': '7-BFI_A', 'Q136_3': '8-BFI_C','Q136_4': '9-BFI_N', 'Q136_5': '10-BFI_O'})

#BFI- questions should be reversed 1,3,4,5,7
raw_onlinefb['1-BFI_E'] = raw_onlinefb['1-BFI_E'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['3-BFI_C'] = raw_onlinefb['3-BFI_C'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['4-BFI_N'] = raw_onlinefb['4-BFI_N'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['5-BFI_O'] = raw_onlinefb['5-BFI_O'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['7-BFI_A'] = raw_onlinefb['7-BFI_A'].map({1:5, 2:4, 3:3, 4:2,5:1})
#to calculate mean of each dimension
# N=[4,9]
# E=[1,6]
# O=[5,10]
# A=[2,7]
# C=[3,8]
#create new cols for calclualtion of each dim of BFI (mean)
bfi_n=['4-BFI_N','9-BFI_N']
raw_onlinefb['BFI_N']=raw_onlinefb[bfi_n].mean(axis=1)
bfi_e=['1-BFI_E','6-BFI_E']
raw_onlinefb['BFI_E']=raw_onlinefb[bfi_e].mean(axis=1)
bfi_o=['5-BFI_O','10-BFI_O']
raw_onlinefb['BFI_O']=raw_onlinefb[bfi_o].mean(axis=1)
bfi_a=['2-BFI_A','7-BFI_A']
raw_onlinefb['BFI_A']=raw_onlinefb[bfi_a].mean(axis=1)
bfi_c=['3-BFI_C','8-BFI_C']
raw_onlinefb['BFI_C']=raw_onlinefb[bfi_c].mean(axis=1)


# In[6]:



#CRT-there are three questions with right answer 
crt_ball=5
crt_machine=5
crt_lake=47
#raw col names
crt_raw=['Q88_1','Q89_1','Q90_1']
#rename the raw cols
raw_onlinefb=raw_onlinefb.rename(columns={'Q88_1':'CRT_ball','Q89_1':'CRT_machine','Q90_1':'CRT_lake'})
#create empty list for three crts
crt_ball=[]
crt_machine=[]
crt_lake=[]
#there are different kind of answers for each crt in raw data
crt_ball_answers=['5','0,05','5 cent']
crt_machine_answers=['5','5 Minuten']
crt_lake_answers=['47','47 Tage']
for i in raw_onlinefb.CRT_ball:
    if i in crt_ball_answers:
        i='True'
        crt_ball.append(i)
    else:
        i='False'
        crt_ball.append(i)
for i in raw_onlinefb.CRT_machine:
    if i in crt_machine_answers:
        i='True'
        crt_machine.append(i)
    else:
        i='False'
        crt_machine.append(i)
for i in raw_onlinefb.CRT_machine:
    if i in crt_machine_answers:
        i='True'
        crt_machine.append(i)
    else:
        i='False'
        crt_machine.append(i)
for i in raw_onlinefb.CRT_lake:
    if i in crt_lake_answers:
        i='True'
        crt_lake.append(i)
    else:
        i='False'
        crt_lake.append(i)
#creating new graphs with results of for loop for each crt question
raw_onlinefb['crt_ball']=crt_ball
raw_onlinefb['crt_machine']=crt_lake
raw_onlinefb['crt_lake']=crt_lake


# In[7]:


#to prepare the STAI qs for data analysis
#For STAI, there are two set of qs, each 20.

#the name of stai cols in raw data
stais_raw=['Q67_1', 'Q67_2', 'Q67_3', 'Q67_4', 'Q67_5', 'Q119_1', 'Q119_2',
 'Q119_3', 'Q119_4', 'Q119_5', 'Q120_1', 'Q120_2', 'Q120_3', 'Q120_4', 'Q120_5',
 'Q121_1', 'Q121_2', 'Q121_3', 'Q121_4', 'Q121_5', 'Q71_1', 'Q71_2', 'Q71_3',
 'Q71_4', 'Q71_5', 'Q122_1', 'Q122_2', 'Q122_3', 'Q122_4', 'Q122_5', 'Q123_1', 'Q123_2',
 'Q123_3', 'Q123_4', 'Q123_5', 'Q124_1', 'Q124_2', 'Q124_3', 'Q124_4', 'Q124_5']
nums=[]
for i in range(1,41):
    i=str(i)+"-"+'stai'
    nums.append(i)
    
#need to synch raw with nums
stai_dic=dict(zip(stais_raw,nums)) 
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q67_1': '1-stai', 'Q67_2': '2-stai', 'Q67_3': '3-stai', 'Q67_4': '4-stai', 'Q67_5': '5-stai',
 'Q119_1': '6-stai', 'Q119_2': '7-stai', 'Q119_3': '8-stai', 'Q119_4': '9-stai', 'Q119_5': '10-stai', 'Q120_1': '11-stai', 'Q120_2': '12-stai',
 'Q120_3': '13-stai', 'Q120_4': '14-stai', 'Q120_5': '15-stai', 'Q121_1': '16-stai', 'Q121_2': '17-stai', 'Q121_3': '18-stai',
 'Q121_4': '19-stai', 'Q121_5': '20-stai', 'Q71_1': '21-stai', 'Q71_2': '22-stai', 'Q71_3': '23-stai', 'Q71_4': '24-stai',
                        'Q71_5': '25-stai', 'Q122_1': '26-stai','Q122_2': '27-stai', 'Q122_3': '28-stai', 'Q122_4': '29-stai', 'Q122_5': '30-stai',
 'Q123_1': '31-stai', 'Q123_2': '32-stai', 'Q123_3': '33-stai', 'Q123_4': '34-stai', 'Q123_5': '35-stai', 'Q124_1': '36-stai',
 'Q124_2': '37-stai', 'Q124_3': '38-stai', 'Q124_4': '39-stai', 'Q124_5': '40-stai'})
#to reverse the specific questions
##There are many questions reverted: Y1=[1,2,5,8,10,11,15,16,19,20] for Y2=[21,23,26,27,30,33,34,36,39]
reversed_qs=['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']
#reverse the answers
raw_onlinefb[['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']] =raw_onlinefb[['1-stai', '2-stai', '5-stai', '8-stai', '10-stai', '11-stai', '15-stai',
 '16-stai', '19-stai', '20-stai', '21-stai', '23-stai', '26-stai', '27-stai', '30-stai',
             '33-stai', '34-stai','36-stai','39-stai']].replace({1:4, 2:3, 3:2, 4:1})
#calculate score of each participant for STAI
raw_onlinefb['stai_total']=raw_onlinefb[reversed_qs].sum(axis=1)


# In[8]:


#to prepare raw data from SDS 17
raw_sds=['Q33', 'Q65', 'Q49', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64']
# new col names
sds_num=[]
for i in range(1,18):
    i=str(i)+"-"+'SDS17'
    sds_num.append(i)
#need to synch raw with nums
sds17_dic=dict(zip(raw_sds,sds_num)) 
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q33': '1-SDS17', 'Q65': '2-SDS17', 'Q49': '3-SDS17', 'Q51': '4-SDS17',
                                          'Q52': '5-SDS17', 'Q53': '6-SDS17', 'Q54': '7-SDS17', 'Q55': '8-SDS17', 
                                          'Q56': '9-SDS17', 'Q57': '10-SDS17',
 'Q58': '11-SDS17', 'Q59': '12-SDS17', 'Q60': '13-SDS17','Q61': '14-SDS17', 
 'Q62': '15-SDS17', 'Q63': '16-SDS17', 'Q64': '17-SDS17'})
#there is no reverse scoring in SDS17 but i should only have the sum of 1 in raw data which means True
sds_cols=['1-SDS17', '2-SDS17', '3-SDS17', '4-SDS17', '5-SDS17', '6-SDS17', '7-SDS17', '8-SDS17', '9-SDS17', '10-SDS17',
 '11-SDS17', '12-SDS17', '13-SDS17', '14-SDS17', '15-SDS17', '16-SDS17','17-SDS17']

raw_onlinefb['sds_total']=raw_onlinefb[raw_onlinefb[sds_cols]==1].sum(axis=1)         


# In[9]:


#to prepare raw data of MEQ
# MEQ scoring:
# Q1->1-6
# Q2->4-1
# Q3->1-5
# Q4->1-5
# Q5->1-4
meq_ls=['Q12', 'Q13', 'Q14', 'Q15', 'Q16']
#change the name of cols
meq_cols=['meq-1','meq-2','meq-3','meq-4','meq-5']
#need to synch raw with nums
meq_dict=dict(zip(meq_ls,meq_cols))
#rename the cols from raw to created ones
raw_onlinefb=raw_onlinefb.rename(columns={'Q12': 'meq-1', 'Q13': 'meq-2', 'Q14': 'meq-3', 'Q15': 'meq-4','Q16':'meq-5'})
#teh second q should be reversed
raw_onlinefb['meq-2'] = raw_onlinefb['meq-2'].map({4:1, 3:2, 2:3, 1:4})
raw_onlinefb['meq_total']=raw_onlinefb[['meq-1','meq-2','meq-3','meq-4','meq-5']].sum(axis=1)
#the higher the number more ecening person

#to prepare escale
raw_cols=['Q138_1', 'Q138_2', 'Q138_3', 'Q138_4', 'Q138_5', 'Q140_1', 'Q140_2', 'Q140_3', 'Q140_4','Q140_5', 'Q141_1',
 'Q141_2', 'Q141_3', 'Q141_4', 'Q141_5', 'Q142_1', 'Q142_2', 'Q142_3', 'Q142_4', 'Q142_5', 'Q143_1', 'Q143_2','Q143_3', 
          'Q143_4', 'Q143_5']
# Factor 1- cognitive-sensitivity [CS] 14,17,19,24,21
# Factor 2- emotional-sensitivity [ES]1,2, 5, 7, 9, 11
# Factor 3- emotional-concern [EC]10, 15, 18, 20 (negatively poled), 22, 23, 25 
# Factor 4- cognitive-concern[CC] 3, 4, 6, 8, 13
cs=[14,17,19,24,21]
es=[1,2, 5, 7, 9, 11]
ec=[10, 15, 18, 20, 22, 23, 25 ]
cc=[3, 4, 6, 8, 13]
e_scale=[]
for i in range(1,26):
    if i in cs:
        i=str(i)+"-"+'escale_cs'
        e_scale.append(i)
    if i in es:
        i=str(i)+"-"+'escale_es'
        e_scale.append(i)
    if i in ec:
        i=str(i)+"-"+'escale_ec'
        e_scale.append(i)
    if i in cc:
        i=str(i)+"-"+'escale_cc'
        e_scale.append(i)
        
escale_dict=dict(zip(raw_cols,e_scale))        
raw_onlinefb=raw_onlinefb.rename(columns={'Q138_1': '1-escale_es', 'Q138_2': '2-escale_es', 
                                          'Q138_3': '3-escale_cc', 'Q138_4': '4-escale_cc', 'Q138_5': '5-escale_es', 'Q140_1': '6-escale_cc',
 'Q140_2': '7-escale_es', 'Q140_3': '8-escale_cc','Q140_4': '9-escale_es', 'Q140_5': '10-escale_ec', 
                                          'Q141_1': '11-escale_es', 'Q141_2': '13-escale_cc',
 'Q141_3': '14-escale_cs', 'Q141_4': '15-escale_ec', 'Q141_5': '17-escale_cs', 'Q142_1': '18-escale_ec',
                                          'Q142_2': '19-escale_cs', 'Q142_3': '20-escale_ec',
 'Q142_4': '21-escale_cs', 'Q142_5': '22-escale_ec', 'Q143_1': '23-escale_ec', 'Q143_2': '24-escale_cs',
                                          'Q143_3': '25-escale_ec'})
raw_onlinefb['20-escale_ec'] = raw_onlinefb['20-escale_ec'].map({1:5, 2:4, 3:3, 4:2,5:1})
raw_onlinefb['e-scale_total']=raw_onlinefb[e_scale].sum(axis=1)



#to remove the empty cols of avatar test and bringing all under each other


#remove the cols from 1_Q194_First Click to 1_Q196_First Click



# Replacing empty string with np.NaN
import numpy as np
#if vp has already fill the qs in deins oder meins drop it! 
raw_onlinefb=raw_onlinefb.loc[raw_onlinefb['Q208'] !=1]
raw_onlinefb = raw_onlinefb.replace('', np.nan)
df_left=raw_onlinefb[~raw_onlinefb['1_Q194_First Click'].isnull()]
#remove the unwanted cols from df_left

df_left.drop(columns=df_left.loc[:,'1_Q196_First Click':'64_Q193'], axis=1,inplace=True)
#creating two separated dfs to get rid of duplicate cols
df_right=raw_onlinefb[raw_onlinefb['1_Q194_First Click'].isnull()]

#remove the unwanted cols from df_right
df_right.drop(columns=df_right.loc[:,'1_Q194_First Click':'64_Q175'], axis=1,inplace=True)

#df_left name of cols 
col_df_left=df_left.loc[:,'1_Q194_First Click':'64_Q175'].columns.to_list()
#df_right name of cols
col_df_right=df_right.loc[:,'1_Q196_First Click':'64_Q193'].columns.to_list()
#make dict for both cols names
dict_col_name=dict(zip(col_df_right,col_df_left))  
#rename the df_right specific col names by df_left col names
df_right=df_right.rename(dict_col_name,axis=1)
#add the information which order of perspective they did: left or right
df_left['perspective-order']='left'
df_right['perspective-order']='right'
#encode the right and wrong questions
#for now I coded not answered 'NONE'qs as wrong answered question
#left order 5=stimmt nicht überein 4=stimmt überein 
#right order 4=stimmt nicht überein , 5=stimmt überein 
#questions and correct answers

#question_left_5 as correct answer
ql5c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_left[ql5c]= np.where(df_left[ql5c]==5, 0, 1)
#question_right_4 as correct answer
qr4c=['1_Q175','5_Q175','7_Q175','8_Q175','13_Q175','14_Q175','18_Q175','19_Q175','21_Q175','22_Q175','23_Q175','30_Q175','31_Q175','35_Q175'
,'36_Q175','38_Q175','39_Q175','40_Q175','41_Q175','43_Q175','46_Q175','48_Q175','51_Q175','52_Q175','53_Q175','55_Q175','56_Q175','57_Q175',
'59_Q175','60_Q175','63_Q175','64_Q175']
df_right[qr4c]= np.where(df_right[qr4c]==4, 0, 1)
#question_left_4 as correct answer
ql4c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_left[ql4c]= np.where(df_left[ql4c]==4, 0, 1)
#question_right_5 as correct answer
qr5c=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','10_Q175','11_Q175','12_Q175','15_Q175','16_Q175','17_Q175','20_Q175','24_Q175','25_Q175'
,'26_Q175','27_Q175','28_Q175','29_Q175','32_Q175','33_Q175','34_Q175','37_Q175','42_Q175','44_Q175','45_Q175','47_Q175','49_Q175','50_Q175',
'54_Q175','58_Q175','61_Q175','62_Q175']
df_right[qr5c]= np.where(df_right[qr5c]==5, 0, 1)
# i need to create four groups 1. self congurent (sc) 2. self incongruent(si) 3.other congruent(oc) 4.other incongruent (oi)
sc=['10_Q175','12_Q175','15_Q175','16_Q175','20_Q175','25_Q175','26_Q175','27_Q175','28_Q175','33_Q175','34_Q175','42_Q175','47_Q175','49_Q175'
,'61_Q175','62_Q175']
si=['1_Q175','7_Q175','14_Q175','18_Q175','21_Q175','31_Q175','36_Q175','38_Q175','41_Q175','46_Q175','51_Q175','56_Q175','59_Q175','60_Q175'
,'63_Q175','64_Q175']
oc=['2_Q175','3_Q175','4_Q175','6_Q175','9_Q175','11_Q175','17_Q175','24_Q175','29_Q175','32_Q175','37_Q175','44_Q175','45_Q175','50_Q175'
,'54_Q175','58_Q175']
oi=['5_Q175','8_Q175','13_Q175','19_Q175','22_Q175','23_Q175','30_Q175','35_Q175','39_Q175','40_Q175','43_Q175','48_Q175','52_Q175','53_Q175'
,'55_Q175','57_Q175']

#append both dfs
df=df_left.append(df_right)
#column for sum of sc
df['self_congurent']= df.apply(lambda row: row[sc].sum(),axis=1)
df['self_congurent']=(16-df['self_congurent'])/16
#column for sum of si
df['self_incongurent']=df.apply(lambda row: row[si].sum(),axis=1)
df['self_incongurent']=(16-df['self_incongurent'])/16
#column for sum of oc
df['other_congurent']=df.apply(lambda row: row[oc].sum(),axis=1)
df['other_congurent']=(16-df['other_congurent'])/16
#column for sum of oi
df['other_incongurent']=df.apply(lambda row: row[oi].sum(),axis=1)
df['other_incongurent']=(16-df['other_incongurent'])/16
#calculate the incongurent! if value is >0 means self incongurent and if <0 other ingurent and 0 means no ingurennt
df['perspective']=(df['self_incongurent']-df['self_congurent'])-(df['other_incongurent']-df['other_congurent'])


# In[11]:


df.rename(columns={'StartDate':'Date','Q132_1':'Vp-Code'},inplace=True)


# In[12]:


#export only necessary cols for statistical analysis
cols_wanted=['Date','Duration (in seconds)','Vp-Code','TICS', 'TICS_control1', 'TICS_control2', 'TICS_control3', 'SVO_Final','BIS_Total_qs', 'BAS_total_qs', 'BAS_drive', 'BAS_FUN', 'BAS_rewards', 'BIS_15_MI', 'BIS_15_NPI', 'BIS_15_ABI',
 'BIS_15_total', 'BFI_N', 'BFI_E', 'BFI_O', 'BFI_A', 'BFI_C', 'crt_ball', 'crt_machine', 'crt_lake', 'stai_total', 'sds_total',
 'meq_total', 'e-scale_total', 'perspective']


# In[13]:


df=df[cols_wanted]


# In[14]:


#export the edited DF
df_append=pd.ExcelWriter('df_online_luca.xlsx')
df.to_excel(df_append)
df_append.save()





/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-36-6e75f4705e78>:803: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left['perspective-order']='left'
<ipython-input-36-6e75f4705e78>:815: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [37]:
#Panas

#import the libraries
import pandas as pd
import numpy as np
#import the raw data
panas_raw=pd.read_excel('panas_raw.xlsx')
panas_raw.rename(columns={'VP-Code':'Vp-Code'},inplace=True)

#remove fake rows
# list the rows, see which one is fake


panas_raw = panas_raw.drop(panas_raw[panas_raw['Vp-Code'].isin(['sdewewa','Karaviti Evdoxia ','BRHA12M','mg1122','TEST','Fake','1234','12345','test','Test'])].index)
panas_raw = panas_raw.drop(panas_raw.index[[0,3]])


#remove unwanted cols
unwanted_cols=[
 'Status', 'IPAddress','sex-sample1', 'Finished',  'Tablet-taken_stress1',
    'RecordedDate', 'RecipientLastName',
 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude',
 'LocationLongitude', 'DistributionChannel','stress3', 'UserLanguage']
panas_raw=panas_raw.drop(columns=unwanted_cols, axis=1)

#change the name of COLS from stress after 40 min
original_cols=['Q15_1',
 'Q15_2', 'Q15_3', 'Q15_4', 'Q15_5', 'Q15_6', 'Q15_7', 'Q15_8', 'Q15_9',
 'Q15_10', 'Q15_11', 'Q15_12', 'Q15_13','Q15_14',
 'Q15_15', 'Q15_16', 'Q15_17', 'Q15_18', 'Q15_19', 'Q15_20']
min_40=['PANAS-40min_1', 'PANAS-40min_2', 'PANAS-40min_3',
 'PANAS-40min_4', 'PANAS-40min_5', 'PANAS-40min_6', 'PANAS-40min_7', 'PANAS-40min_8', 
'PANAS-40min_9', 'PANAS-40min_10', 'PANAS-40min_11', 'PANAS-40min_12',
 'PANAS-40min_13', 'PANAS-40min_14', 'PANAS-40min_15',
 'PANAS-40min_16', 'PANAS-40min_17', 'PANAS-40min_18', 'PANAS-40min_19', 'PANAS-40min_20']

dict_panas=dict(zip(original_cols,min_40))
panas_raw = panas_raw.rename(columns=dict_panas)

#panas_raw.columns.to_list()
#list of positive items
#positive=[1,3,5,9,10,12,14,16,17,19]
#negative=[2,4,6,7,8,11,13,15,18,20]
##for PANAS in base
#positive items
positive_basic=['PANAS-basic_1','PANAS-basic_3','PANAS-basic_5','PANAS-basic_9',
               'PANAS-basic_10','PANAS-basic_12','PANAS-basic_14','PANAS-basic_16',
               'PANAS-basic_17','PANAS-basic_19']
#negative items
negative_basic=['PANAS-basic_2','PANAS-basic_11','PANAS-basic_13','PANAS-basic_15',
               'PANAS-basic_8','PANAS-basic_12','PANAS-basic_14','PANAS-basic_16',
               'PANAS-basic_18','PANAS-basic_20']
#new col by calculating sum of positive and negative iitems
panas_raw['positive_basic']=panas_raw[positive_basic].sum(axis=1)
panas_raw['negative_basic']=panas_raw[negative_basic].sum(axis=1)


#for PANAS after 20 minutes
positive_20min=['PANAS-20min_1','PANAS-20min_3','PANAS-20min_5','PANAS-20min_9',
               'PANAS-20min_10','PANAS-20min_12','PANAS-20min_14','PANAS-20min_16',
               'PANAS-20min_17','PANAS-20min_19']
negative_20min=['PANAS-20min_2','PANAS-20min_11','PANAS-20min_13','PANAS-20min_15',
               'PANAS-20min_8','PANAS-20min_12','PANAS-20min_14','PANAS-20min_16',
               'PANAS-20min_18','PANAS-20min_20']
#new col by calculating sum of positive and negative iitems
panas_raw['positive_20min']=panas_raw[positive_20min].sum(axis=1)
panas_raw['negative_20min']=panas_raw[negative_20min].sum(axis=1)

#for PANAS after 40 minutes
positive_40min=['PANAS-40min_1','PANAS-40min_3','PANAS-40min_5','PANAS-40min_9',
               'PANAS-40min_10','PANAS-40min_12','PANAS-40min_14','PANAS-40min_16',
               'PANAS-40min_17','PANAS-40min_19']
negative_40min=['PANAS-40min_2','PANAS-40min_11','PANAS-40min_13','PANAS-40min_15',
               'PANAS-40min_8','PANAS-40min_12','PANAS-40min_14','PANAS-40min_16',
               'PANAS-40min_18','PANAS-40min_20']
#new col by calculating sum of positive and negative iitems
panas_raw['positive_40min']=panas_raw[positive_40min].sum(axis=1)
panas_raw['negative_40min']=panas_raw[negative_40min].sum(axis=1)

#adding cols of stress levels
panas_raw['basic_stress_level']=panas_raw['stress-basic_1']
panas_raw['20min_stress_level']=panas_raw['stress-20min_1']
panas_raw['40_stress_level']=panas_raw['stress-40min_1']
#drop empty cel


panas_raw['Vp-Code']=panas_raw['Vp-Code'].dropna()


#export the whole panas df
df_panas=pd.ExcelWriter('df_panas.xlsx')
panas_raw.to_excel(df_panas)
df_panas.save()


panas_raw.columns.to_list()
selected_cols=['StartDate', 'EndDate', 'Progress', 'Duration (in seconds)',
 'ResponseId','Vp-Code', 'positive_basic', 'negative_basic',
 'positive_20min', 'negative_20min', 'positive_40min',
 'negative_40min', 'basic_stress_level', '20min_stress_level', '40_stress_level']
selected_panas=panas_raw[selected_cols]
#since first two VPs did not have panas, I should add their rows
#all with no data. To have consistent data in all dataframes.
ANHA19E={'StartDate':'NO_DATA', 'EndDate':'NO_DATA' ,'Progress':'NO_DATA',
         'Duration (in seconds)':'NO_DATA',
 'ResponseId':'NO_DATA','Vp-Code':'ANHA19E', 'positive_basic':'NO_DATA',
         'negative_basic':'NO_DATA',
 'positive_20min':'NO_DATA', 'negative_20min':'NO_DATA', 'positive_40min':'NO_DATA',
 'negative_40min':'NO_DATA', 'basic_stress_level':'NO_DATA',
         '20min_stress_level':'NO_DATA', '40_stress_level':'NO_DATA'}
PASS13D={'StartDate':'NO_DATA', 'EndDate':'NO_DATA' ,
         'Progress':'NO_DATA','Duration (in seconds)':'NO_DATA',
 'ResponseId':'NO_DATA','Vp-Code':'PASS13D','positive_basic':'NO_DATA',
         'negative_basic':'NO_DATA','positive_20min':'NO_DATA', 
         'negative_20min':'NO_DATA', 'positive_40min':'NO_DATA',
 'negative_40min':'NO_DATA', 'basic_stress_level':'NO_DATA',
         '20min_stress_level':'NO_DATA', '40_stress_level':'NO_DATA'}

selected_panas=panas_raw[selected_cols]
#first VP
selected_panas=selected_panas.append(ANHA19E,ignore_index=True)
#second VP
selected_panas=selected_panas.append(PASS13D,ignore_index=True)
selected_panas['Vp-Code'].dropna(how='any',inplace=True)
sel_panas=pd.ExcelWriter('sel_panas.xlsx')
selected_panas.to_excel(sel_panas)
sel_panas.save()




In [38]:
#DOM Tasks


df_raw_dom=pd.read_excel('Dom_raw.xlsx')

#cleaning->removing unwanted rows and cols
#row
df_raw_dom=df_raw_dom.drop(df_raw_dom.index[0])
#cols
df_raw_dom=df_raw_dom.drop(columns=['Status', 'IPAddress',
 'Finished', 'RecordedDate',
 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'], axis=1)
#rename the vp code col
df_raw_dom=df_raw_dom.rename(columns={'Q1':'Vp-Code'})
#drop unwanted rows
df_raw_dom = df_raw_dom.drop(df_raw_dom[df_raw_dom['Vp-Code'].isin(['Fake','FAKE','1234','12345','test','TEST'])].index)
df_raw_dom = df_raw_dom.drop(df_raw_dom.index[0])
df_raw_dom=df_raw_dom.drop(columns=['Q_RecaptchaScore'], axis=1)


df_sd=df_raw_dom.loc[:,'StartDate':'Give_100']



df_iat = df_raw_dom[list(df_raw_dom.columns[0:6]) + list(df_raw_dom.columns[34:44])]


#calculating blatant and subtle prejudice
blatant_cols=['Threat-Rejection-1','Threat-Rejection-2', 
              'Threat-Rejection-3', 'Threat-Rejection-4',
              'Threat-Rejection-5', 'Threat-Rejection-6']

anti_intimacy_cols=['Intimacy Items_1', 'Intimacy Items_2', 'Intimacy Items_3',
 'Intimacy Items_4']
blatant_dict={'Stimme voll zu':4,'Stimme teilweise zu':3,
             'Weder noch':2,'Stimme  teilweise nicht zu':1,
             'Stimme überhaupt nicht zu':0}
#replace str with int
df_iat.loc[:,'Threat-Rejection-1':'Threat-Rejection-6'].replace({'Stimme voll zu':4,'Stimme teilweise zu':3,
             'Weder noch':2,'Stimme  teilweise nicht zu':1,'Stimme teilweise nicht zu':1,
             'Stimme überhaupt nicht zu':0},inplace=True)
#replace str with int
df_iat.loc[:,'Intimacy Items_1':'Intimacy Items_4'].replace({'Stimme voll zu':4,'Stimme teilweise zu':3,
                                                             'Weder noch':2,'Stimme teilweise nicht zu':1,
                                                            'Stimme überhaupt nicht zu':0},inplace=True)
#the first question of anti_intimacy should stay as it is! 
#these questions should be reversed.
df_iat['Intimacy Items_2']=df_iat['Intimacy Items_2'].map({4:1,3:2,2:3,1:4})
df_iat['Intimacy Items_3']=df_iat['Intimacy Items_3'].map({4:1,3:2,2:3,1:4})
df_iat['Intimacy Items_4']=df_iat['Intimacy Items_4'].map({4:1,3:2,2:3,1:4})
#adding a new col, total of each balant and anti intimacy items
df_iat['blatant_total']=df_iat[blatant_cols].sum(axis=1)
df_iat['anti_intimacy_total']=df_iat[anti_intimacy_cols].sum(axis=1)
#remove the rows in vp_code that are test or by fake VPs
not_wanted_rows=['Fake','fake','test','TEST','FAKE','Test','richard'
                ,'Evdoxia','Evdoxia Karaviti']
df_iat=df_iat[~df_iat['Vp-Code'].isin(not_wanted_rows)]
#to add the missing VP code of first 5 rows
df_iat.at[2,'Vp-Code']='ANHA19E'
df_iat.at[3,'Vp-Code']='PAAS13D'
df_iat.at[4,'Vp-Code']='GAJO22B'
df_iat.at[5,'Vp-Code']='TAST22E'
df_iat.at[6,'Vp-Code']='Heto16B'
#drop the duplicate HERA27K
df_iat.drop(12,axis=0, inplace=True)

df_iat.sort_values(by=['ResponseId'], inplace=True)
#import the iat from iatgen which is in csv
iatgen=pd.read_csv('iat-results-3.csv',keep_default_na=False)

iatgen.rename(columns={'ResponseID':'ResponseId'},inplace=True)
iatgen.sort_values(by=['ResponseId'], inplace=True)

df_iat=df_iat.merge(iatgen, on='ResponseId')
#drop the unwanted col from iatgen df
df_iat.drop(columns=['idid'],axis=1,inplace=True)

#export the edited iat_DF
df_iat1=pd.ExcelWriter('df_iat.xlsx')
df_iat.to_excel(df_iat1)
df_iat1.save()

#to clean and export the IPD-MD task data
df_ipd = df_raw_dom[list(df_raw_dom.columns[0:6]) + list(df_raw_dom.columns[66:80])]
ipd_change_col_name=['10-euro_1', '10-euro_2', '10-euro_3',
 'Q38' ,'Q39', '8-euro_1', '8-euro_2', '8-euro_3', 'Q40',
 'Q42', '5-euro_1', '5-euro_2', '5-euro_3', 'Q41', 'Q43']
ipd_new_name=['no_lost_A','no_lost_B','no_lost_C','no_lost_ingroup',
              'no_lost_outgroup','20%_lost_A','20%_lost_B','20%_lost_C',
              '20%_lost_ingroup','20%_lost_outgroup','50%_lost_A','50%_lost_B','50%_lost_C','50%_lost_ingroup',
              '50%_lost_outgroup']
dict_ipd=dict(zip(ipd_change_col_name,ipd_new_name))
df_ipd=df_ipd.rename(columns=dict_ipd)
#to add the missing VP code of first 5 rows
df_ipd.at[2,'Vp-Code']='ANHA19E'
df_ipd.at[3,'Vp-Code']='PAAS13D'
df_ipd.at[4,'Vp-Code']='GAJO22B'
df_ipd.at[5,'Vp-Code']='TAST22E'
df_ipd.at[6,'Vp-Code']='Heto16B'
#drop the duplicate HERA27K
df_ipd.drop(12,axis=0, inplace=True)
#remove the rows in vp_code that are test or by fake VPs
not_wanted_rows=['Fake','fake','test','TEST','FAKE','Test','richard'
                ,'Evdoxia','Evdoxia Karaviti']
df_ipd=df_ipd[~df_ipd['Vp-Code'].isin(not_wanted_rows)]

#calculate the percent of moey for each condition of IPD
lost_no=['no_lost_A','no_lost_B','no_lost_C']
lost_20=['20%_lost_A','20%_lost_B','20%_lost_C']
lost_50=['50%_lost_A','50%_lost_B','50%_lost_C']
df_ipd[lost_no]=df_ipd[lost_no]/10*100
df_ipd[lost_20]=df_ipd[lost_20]/8*100
df_ipd[lost_50]=df_ipd[lost_50]/5*100

#export the edited IPDMD
df_ipd_excel=pd.ExcelWriter('df_ipd.xlsx')
df_ipd.to_excel(df_ipd_excel)
df_ipd_excel.save()

#to clean and export the demographical data
df_demog= df_raw_dom[list(df_raw_dom.columns[0:6]) + list(df_raw_dom.columns[104:])]
#change the name of cols [ q7 and q8]
df_demog.rename(columns={'Q7':'Sex','Q8':'Age'},inplace=True)
#to add the missing VP code of first 5 rows
df_demog.at[2,'Vp-Code']='ANHA19E'
df_demog.at[3,'Vp-Code']='PAAS13D'
df_demog.at[4,'Vp-Code']='GAJO22B'
df_demog.at[5,'Vp-Code']='TAST22E'
df_demog.at[6,'Vp-Code']='Heto16B'
#remove the rows in vp_code that are test or by fake VPs
not_wanted_rows=['Fake','fake','test','TEST','FAKE','Test','richard'
                ,'Evdoxia','Evdoxia Karaviti']
df_demog=df_demog[~df_demog['Vp-Code'].isin(not_wanted_rows)]

#change the cols of demographic qs to text 
#q1
d1=[]
for i in df_demog['Debriefing-Q1']:
    if i==1:
        i='Placebo'
        d1.append(i)
    else:
        i='Drug'
        d1.append(i)
df_demog['Debriefing-Q1']=d1
#q2
df_demog['Debriefing-Q2'].fillna(value=5,inplace=True)
d2=[]
for i in df_demog['Debriefing-Q2']:
    if i==1:
        i='Hydrocortison'
        d2.append(i)
    if i==2:
        i='Yohimbine'
        d2.append(i)
    if i==3:
        i='H+Y'
        d2.append(i)
    if i==4:
        i='Placebo'
        d2.append(i)
    if i==5:
        i='No-info'
        d2.append(i)
        
df_demog['Debriefing-Q2']=d2
df_demog.rename(columns={'Debriefing-Q2':'Which_drug?'},inplace=True)
#q4
d4=[]
for i in df_demog['Debriefing-Q4']:
    if i==1:
        i='YES'
        d4.append(i)
    else:
        i='No'
        d4.append(i)
df_demog['Debriefing-Q4']=d4
df_demog.rename(columns={'Debriefing-Q4':'effect_on_your_behavior?'},inplace=True)
#'Demo_nationality'
#there is one nan that I will change to 6 which means no info
df_demog['Demo_nationality'].fillna(value=6,inplace=True)
nationality=[]
for i in df_demog['Demo_nationality']:
    if i==1:
        i='German'
        nationality.append(i)
    if i==2:
        i='Turkish'
        nationality.append(i)
    if i==3:
        i='Polnish'
        nationality.append(i)
    if i==4:
        i='Syrian'
        nationality.append(i)
    if i==5:
        i='Other'
        nationality.append(i)
    if i==6:
        i='No_Info'
        nationality.append(i)
df_demog['Demo_nationality']=nationality
#education
#there is one nan that I will change to 6 which means no info
df_demog['Demo_education'].fillna(value=9,inplace=True)
education=[]
for i in df_demog['Demo_education']:
    if i==1:
        i='primary-incomplete'
        education.append(i)
    if i==2:
        i='primary-complete'
        education.append(i)
    if i==3:
        i='middle school-incomplete'
        education.append(i)
    if i==4:
        i='middle-school-complete'
        education.append(i)
    if i==5:
        i='technical student'
        education.append(i)
    if i==6:
        i='uni student'
        education.append(i)
    if i==7:
        i='Bachelor'
        education.append(i)
    if i==8:
        i='master_or_phd'
        education.append(i)
    if i==9:
        i='No_info'
        education.append(i)
df_demog['Demo_education']=education
#income
df_demog['Demo_income'].fillna(value=6,inplace=True)
income=[]
for i in df_demog['Demo_income']:
    if i==1:
        i='<25.000'
        income.append(i)
    if i==2:
        i='25.000-50.000'
        income.append(i)
    if i==3:
        i='50.000-100.000'
        income.append(i)
    if i==4:
        i='100.000-200.000'
        income.append(i)
    if i==5:
        i='>200.000'
        income.append(i)
    if i==6:
        i='No_Info'
        income.append(i)
df_demog['Demo_income']=income
#employment
df_demog['Demo_employment'].fillna(value=6,inplace=True)
employment=[]
for i in df_demog['Demo_employment']:
    if i==1:
        i='working'
        employment.append(i)
    if i==2:
        i='student'
        employment.append(i)
    if i==3:
        i='job searcher'
        employment.append(i)
    if i==4:
        i='other'
        employment.append(i)
    if i==5:
        i='no-info'
        employment.append(i)
    if i==6:
        i='student'
        employment.append(i)
    if i==7:
        i='no-info'
        employment.append(i)
    if i==8:
        i='no-info'
employment.append(i)
df_demog['Demo_employment']=employment
#Q7
df_demog['Sex'].fillna(value=4,inplace=True)
sex1=[]
for i in df_demog['Sex']:
    if i==1:
        i='F'
        sex1.append(i)
    if i==2:
        i='M'
        sex1.append(i)
    if i==3:
        i='Other'
        sex1.append(i)
    if i==4:
        i='no-info'
        sex1.append(i)
df_demog['Sex']=sex1
df_demog.drop(12,axis=0, inplace=True)
df_demog_xls=pd.ExcelWriter('df_demog_xls.xlsx')
df_demog.to_excel(df_demog_xls)
df_demog_xls.save()

#to create the data of social discounting. there are one social discounting 
#from september which is for the previous version of the sd task
sd_sep=pd.read_excel('Copy of YOM_7 September 2022_13.30.xlsx')
#the data from the new version of the sd task
sd_new=pd.read_excel('Dom_raw.xlsx')
#columns of sd data
sd_columns=['ResponseId','StartDate','EndDate','Q1','Taken_1', 'Taken_2', 'Taken_3', 'Taken_5',
 'Taken_10 ', 'Taken_20', 'Taken_50 ', 'Taken_100', 'Give_1 ',
 'Give_2', 'Give_3', 'Give_5', 'Give_10', 'Give_20', 'Give_50', 'Give_100']
sd_sep=sd_sep[sd_columns]
sd_sep=sd_sep.drop(index=0)
sd_sep=sd_sep.rename(columns={'Q1':'Vp-Code'})
sd_sep=sd_sep[~sd_sep['Vp-Code'].isin(not_wanted_rows)]
sd_new=sd_new[sd_columns]
sd_new=sd_new.drop(index=range(0,32))
sd_new=sd_new.rename(columns={'Q1':'Vp-Code'})
not_wanted_rows=['Fake','fake','test','TEST','FAKE','Test','richard'
                ,'Evdoxia','Evdoxia Karaviti']
sd_new=sd_new[~sd_new['Vp-Code'].isin(not_wanted_rows)]
#append two dfs
sd_df=sd_sep.append(sd_new)
sd_df=sd_df.drop(index=[1,12])
#adding the vp names to those did not have. because at first days we did
#not ask vps to enter their vps codes in DOM survey
sd_df.at[2,'Vp-Code']='ANHA19E'
sd_df.at[3,'Vp-Code']='PAAS13D'
sd_df.at[4,'Vp-Code']='GAJO22B'
sd_df.at[5,'Vp-Code']='TAST22E'
sd_df.at[6,'Vp-Code']='Heto16B'


df_sd1=pd.ExcelWriter('df_sd.xlsx')
sd_df.to_excel(df_sd1)
df_sd1.save()


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-38-19d976a50a36>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iat['Intimacy Items_2']=df_iat['Intimacy Items_2'].map({4:1,3:2,2:3,1:4})
<ipython-input-38-19d976a50a36>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [39]:
#Mergeing all datasets
import pandas as pd
#first concat dfs from two online QS
online_dom=pd.read_excel('df_online_DOM.xlsx')
online_luca=pd.read_excel('df_online_luca.xlsx')
df_dom_luca=[online_dom,online_luca]
online_df=pd.concat(df_dom_luca)
#drop the vps that are duplicate
online_df.drop_duplicates(subset=['Vp-Code'], inplace=True)
#drop the vps of tests
test_vps=['Finja']
online_df=online_df[online_df['Vp-Code'].isin(test_vps)==False]
#There is no Online Data for this VP
MAPI19B={'Unnamed: 0':'No_DATA', 'Unnamed: 0.1':'No_DATA', 'Date':'No_DATA', 'Duration (in seconds)':'No_DATA', 'Vp-Code':'MAPI29B',
 'TICS':'No_DATA', 'TICS_control1':'No_DATA', 'TICS_control2':'No_DATA', 'TICS_control3':'No_DATA', 'SVO_Final':'No_DATA', 'BIS_Total_qs':'No_DATA', 'BAS_total_qs':'No_DATA',
 'BAS_drive':'No_DATA', 'BAS_FUN':'No_DATA', 'BAS_rewards':'No_DATA', 'BIS_15_MI':'No_DATA', 'BIS_15_NPI':'No_DATA', 'BIS_15_ABI':'No_DATA', 'BIS_15_total':'No_DATA', 
         'BFI_N':'No_DATA', 'BFI_E':'No_DATA', 'BFI_O':'No_DATA', 'BFI_A':'No_DATA', 'BFI_C':'No_DATA', 'crt_ball':'No_DATA', 'crt_machine':'No_DATA',
         'crt_lake':'No_DATA', 'stai_total':'No_DATA', 'sds_total':'No_DATA', 'meq_total':'No_DATA',
 'e-scale_total':'No_DATA', 'perspective':'No_DATA'}

online_df=online_df.append(MAPI19B,ignore_index=True)
#export the edited DF
df_concat=pd.ExcelWriter('df_online_concat.xlsx')
online_df.to_excel(df_concat)
df_concat.save()

#first I will merge four DFs of with this order: PANAS-IPD-IAT-SD-Demog
import pandas as pd
from functools import reduce
#data of online QS
online_df=pd.read_excel('df_online_concat.xlsx')
#data of stress+panas
panas=pd.read_excel('sel_panas.xlsx',keep_default_na=False)
#sort based on by col!
panas.sort_values(by=['Vp-Code'], inplace=True)
#data of IPD-MD task
ipd=pd.read_excel('df_ipd.xlsx')

#sort based on by col!
ipd.sort_values(by=['Vp-Code'], inplace=True)
#data of social discounting task
sd=pd.read_excel('df_sd.xlsx')
#sort based on by col!
sd.sort_values(by=['Vp-Code'], inplace=True)
#data of iat task
iat=pd.read_excel('df_iat.xlsx')
#sort based on by col!
iat.sort_values(by=['Vp-Code'], inplace=True)
#merge fourt DFs. name of df is based in first letter of each df
pisi=[panas,ipd,sd,iat]

#as there is not exact values in Vp-code col, I need to merge them
#multiple times
from fuzzywuzzy import process
# create a choice list
choices = panas['Vp-Code'].values.tolist()

# apply fuzzywuzzy to each row using lambda expression
sd['Vp-Code'] = sd['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
sd[['Vp-Code', 'Percent Match']] = sd['Vp-Code'].apply(pd.Series)

# merge
sd_pa=sd.merge(panas, left_on='Vp-Code', right_on='Vp-Code')

#now merge ipd to the above already merged df
choices = ipd['Vp-Code'].values.tolist()

# apply fuzzywuzzy to each row using lambda expression
sd_pa['Vp-Code'] = sd_pa['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
sd_pa[['Vp-Code', 'Percent Match']] = sd_pa['Vp-Code'].apply(pd.Series)


# merge
sd_pa_ipd=sd_pa.merge(ipd, left_on='Vp-Code', right_on='Vp-Code')

#now merge iat to the above already merged df
choices = iat['Vp-Code'].values.tolist()

# apply fuzzywuzzy to each row using lambda expression
sd_pa_ipd['Vp-Code'] = sd_pa_ipd['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
sd_pa_ipd[['Vp-Code', 'Percent Match']] = sd_pa_ipd['Vp-Code'].apply(pd.Series)


# merge
sd_pa_ipd_iat=sd_pa_ipd.merge(iat, left_on='Vp-Code', right_on='Vp-Code')

#add the data of online questionnires (concat one)
choices = online_df['Vp-Code'].values.tolist()

sd_pa_ipd_iat['Vp-Code'] = sd_pa_ipd_iat['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
sd_pa_ipd_iat[['Vp-Code', 'Percent Match']] = sd_pa_ipd_iat['Vp-Code'].apply(pd.Series)


# merge
online_sd_pa_ipd_iat=sd_pa_ipd_iat.merge(online_df, left_on='Vp-Code', right_on='Vp-Code')

online_sd_pa_ipd_iat = online_sd_pa_ipd_iat[online_sd_pa_ipd_iat['Percent Match']>75]
#drop the cols that are not necessary
online_sd_pa_ipd_iat=online_sd_pa_ipd_iat.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0_x',
                                                       'Unnamed: 0_y','Unnamed: 0.1.1'], axis=1)

import pandas as pd
import numpy as np
df_samples_raw=pd.read_excel('DOM-Samples-Raw.xlsx')
#group by data to have hearth beat for each six times of measurements
df_samples_raw=df_samples_raw.pivot(index=['Vp-Code','Condition','Gender'], columns=['Samples_order'], values='Heartbeat')

#convert pivot table to dataframe
df_samples_raw=df_samples_raw.reset_index()
#change the order of columns to make the sample orders righ
cols=['Vp-Code', 'Condition','Base', '20_minute', '40_minute',  'First_task', 'Second_task', 'Third_taks','Gender']
df_samples_raw=df_samples_raw[cols]
#df_samples_raw=df_samples_raw.rename({'Base':'HB-Base', '20_minute':'HB-20_minute', '40_minute':'HB-40_minute', 
                                     # 'First_task':'HB-First_task','Second_task':'HB-Second_task', 'Third_taks':'HB-Third_taks'})
#sort the data on Vp-Code
df_samples_raw.sort_values(by=['Vp-Code'], inplace=True)
#so far I have not added the sex and stress samples results
#need to add the samples data to combined df
#add the data of samples
choices = df_samples_raw['Vp-Code'].values.tolist()

online_sd_pa_ipd_iat['Vp-Code'] = online_sd_pa_ipd_iat['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
online_sd_pa_ipd_iat[['Vp-Code', 'Percent Match']] = online_sd_pa_ipd_iat['Vp-Code'].apply(pd.Series)


# merge
 
Dom_final=online_sd_pa_ipd_iat.merge(df_samples_raw, left_on='Vp-Code', right_on='Vp-Code')


#add df of sex and stress samples analysis
sex=pd.read_excel('sex_stress samples.xlsx', sheet_name='sex')
stress=pd.read_excel('sex_stress samples.xlsx', sheet_name='stress')
#drop unwanted cols
wcols=['participant code', 'Testosterone pg/ml', 'Progesterone pg/ml', 'Estradiol pg/ml']
sex=sex[wcols]
#rename the col
sex=sex.rename(columns={'participant code':'Vp-Code'})

#sort the data on vp code
sex.sort_values(by=['Vp-Code'], inplace=True)
#pivot table to show eacg time of stress sample analysis separately
df_stress=stress.pivot(index=['Vp-Code','Condition'], columns=['Samples_order'], values=['Yohimbine','hydrocortisone'])
#convert pivot table to dataframe
df_stress=df_stress.reset_index()
df_stress.sort_values(by=['Vp-Code'], inplace=True)
#drop the condition col
df_stress.drop(columns=('Condition'),inplace=True)
#sort the on vp code

#first merge sex sample
choices = sex['Vp-Code'].values.tolist()

Dom_final['Vp-Code'] = Dom_final['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
Dom_final[['Vp-Code', 'Percent Match']] = Dom_final['Vp-Code'].apply(pd.Series)


# merge
 
Dom_final=Dom_final.merge(sex, left_on='Vp-Code', right_on='Vp-Code')#

#second merge the stress sample analysis
choices = df_stress['Vp-Code'].values.tolist()

Dom_final['Vp-Code'] = Dom_final['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
Dom_final[['Vp-Code', 'Percent Match']] = Dom_final['Vp-Code'].apply(pd.Series)
# merge
Dom_final=Dom_final.merge(df_stress, left_on='Vp-Code', right_on='Vp-Code')#
#add the demographical output

choices = df_demog['Vp-Code'].values.tolist()

Dom_final['Vp-Code'] = Dom_final['Vp-Code'].apply(lambda x: process.extractOne(x, choices))
# add percent match wiht apply
Dom_final[['Vp-Code', 'Percent Match']] = Dom_final['Vp-Code'].apply(pd.Series)
# merge
Dom_final=Dom_final.merge(df_demog, left_on='Vp-Code', right_on='Vp-Code')#
#export the edited DF which include all data from all tasks and questionnaires

Dom_final.drop(columns=['Progress_y',"Duration (in seconds)",'Demo_employment_7_TEXT'],inplace=True)
df_concat=pd.ExcelWriter('opisi.xlsx')
Dom_final.to_excel(df_concat)
df_concat.save()



/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
